In [1]:
#connexion avec une base de données PostgreSQL 
import psycopg2
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import re

try:
    conn = psycopg2.connect(
          user = "postgres",
          password = "efrei888",
          host = "localhost",
          port = "5432",
          database = "Beevora"
    )
    cur = conn.cursor()
    # Afficher la version de PostgreSQL 
    cur.execute("SELECT version();")
    version = cur.fetchone()
    print("Version : ", version,"\n")
    
    #fermeture de la connexion à la base de données
    #cur.close()
    #conn.close()
    #print("La connexion PostgreSQL est fermée")

except (Exception, psycopg2.Error) as error :
    print ("Erreur lors de la connexion à PostgreSQL", error)

Version :  ('PostgreSQL 13.4, compiled by Visual C++ build 1914, 64-bit',) 



In [2]:
cur.execute("select match, v_odds.date, v_odds.tournament ->> 'id' as tournament_id, tournaments.category as category, tournaments.name, label, field_name, field_label, \
	(winner -> 0) ->> 'id' as winner, odd, matches.teams[1] as team_1, \
	(select name from teams where id = matches.teams[1]) as name1, matches.teams[2] as team_2, \
	(select name from teams where id = matches.teams[2]) as name2, result #>'{Score,0}'->>'value' as final_score, \
	v_odds.bookmaker_id \
from v_odds, matches, tournaments \
where label = 'Résultat du match' \
and matches.id = v_odds.match \
and tournaments.id = (v_odds.tournament ->> 'id')::int \
and v_odds.sport = 1 \
and odd < 20 ")

res = cur.fetchall()
dfall = pd.DataFrame(res)
dfall.columns = ['match', 'date', 'tournament_id', 'category', 'tournament_name', 'label', 'field_name', 'field_label', 'winner', 'odd', 'team1', 'name1', 'team2', 'name2', 'final_score', 'bookmaker']

In [3]:
dfall

,match,date,tournament_id,category,tournament_name,label,field_name,field_label,winner,odd,team1,name1,team2,name2,final_score,bookmaker
0,28624054,2021-09-30 23:30:00+02:00,27665,49,Primera Division,Résultat du match,Home,Colo Colo,3155,1.27,3155.0,Colo Colo,7029.0,Nublense,0:1,8072
1,25822364,2021-09-29 14:00:00+02:00,704,270,Erovnuli Liga,Résultat du match,Home,FC Samgurali Tskaltubo,44217,1.81,44217.0,FC Samgurali Tskaltubo,5419.0,Torpedo Koutaissi,0:0,8072
2,25406638,2021-08-10 18:00:00+02:00,178,92,Premium Liiga,Résultat du match,Home,Flora Tallinn,3346,1.08,3346.0,Flora Tallinn,3343.0,Trans Narva,5:2,30238
3,28624054,2021-09-30 23:30:00+02:00,27665,49,Primera Division,Résultat du match,Home,Colo Colo,3155,1.35,3155.0,Colo Colo,7029.0,Nublense,0:1,-30239
4,28936218,2021-09-30 18:45:00+02:00,679,393,UEFA Ligue Europa,Résultat du match,Home,Fenerbahce,3052,2.45,3052.0,Fenerbahce,3245.0,Olympiakos,0:3,-8072
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282193,27772134,2021-11-06 18:50:00+01:00,266,66,Première division,Résultat du match,Away,Beitar Jerusalem FC,5204,3.30,5333.0,Maccabi Petah Tikva FC,5204.0,Beitar Jerusalem FC,0:3,-8072
282194,27663880,2021-11-20 15:00:00+01:00,211,23,Superligue,Résultat du match,Away,Spartak Trnava,2406,1.56,7084.0,FC Vion Zlate Moravce - Vrable,2406.0,Spartak Trnava,0:0,6207
282195,25720750,2021-11-07 22:30:00+01:00,240,165,LigaPro Primera A,Résultat du match,Away,SC Delfin,241468,5.00,5257.0,LDU Quito,241468.0,SC Delfin,1:1,6207
282196,25720750,2021-11-07 22:30:00+01:00,240,165,LigaPro Primera A,Résultat du match,Away,SC Delfin,241468,4.90,5257.0,LDU Quito,241468.0,SC Delfin,1:1,9703


In [4]:
dfall['date'] = dfall['date'].map(pd.to_datetime)

def get_y(dt): 
    return dt.year 

def get_moy(dt): 
    return dt.month 

def get_dow(dt): 
    return dt.weekday() + 1

def get_dom(dt): 
    return dt.day 

def get_hour(dt): 
    return dt.hour - 1 

dfall['year'] = dfall['date'].map(get_y)

dfall['month'] = dfall['date'].map(get_moy)

dfall['day_of_week'] = dfall['date'].map(get_dow)

dfall['day_of_month'] = dfall['date'].map(get_dom)

dfall['hour'] = dfall['date'].map(get_hour)

In [5]:
dfall

,match,date,tournament_id,category,tournament_name,label,field_name,field_label,winner,odd,...,name1,team2,name2,final_score,bookmaker,year,month,day_of_week,day_of_month,hour
0,28624054,2021-09-30 23:30:00+02:00,27665,49,Primera Division,Résultat du match,Home,Colo Colo,3155,1.27,...,Colo Colo,7029.0,Nublense,0:1,8072,2021,9,4,30,22
1,25822364,2021-09-29 14:00:00+02:00,704,270,Erovnuli Liga,Résultat du match,Home,FC Samgurali Tskaltubo,44217,1.81,...,FC Samgurali Tskaltubo,5419.0,Torpedo Koutaissi,0:0,8072,2021,9,3,29,13
2,25406638,2021-08-10 18:00:00+02:00,178,92,Premium Liiga,Résultat du match,Home,Flora Tallinn,3346,1.08,...,Flora Tallinn,3343.0,Trans Narva,5:2,30238,2021,8,2,10,17
3,28624054,2021-09-30 23:30:00+02:00,27665,49,Primera Division,Résultat du match,Home,Colo Colo,3155,1.35,...,Colo Colo,7029.0,Nublense,0:1,-30239,2021,9,4,30,22
4,28936218,2021-09-30 18:45:00+02:00,679,393,UEFA Ligue Europa,Résultat du match,Home,Fenerbahce,3052,2.45,...,Fenerbahce,3245.0,Olympiakos,0:3,-8072,2021,9,4,30,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282193,27772134,2021-11-06 18:50:00+01:00,266,66,Première division,Résultat du match,Away,Beitar Jerusalem FC,5204,3.30,...,Maccabi Petah Tikva FC,5204.0,Beitar Jerusalem FC,0:3,-8072,2021,11,6,6,17
282194,27663880,2021-11-20 15:00:00+01:00,211,23,Superligue,Résultat du match,Away,Spartak Trnava,2406,1.56,...,FC Vion Zlate Moravce - Vrable,2406.0,Spartak Trnava,0:0,6207,2021,11,6,20,14
282195,25720750,2021-11-07 22:30:00+01:00,240,165,LigaPro Primera A,Résultat du match,Away,SC Delfin,241468,5.00,...,LDU Quito,241468.0,SC Delfin,1:1,6207,2021,11,7,7,21
282196,25720750,2021-11-07 22:30:00+01:00,240,165,LigaPro Primera A,Résultat du match,Away,SC Delfin,241468,4.90,...,LDU Quito,241468.0,SC Delfin,1:1,9703,2021,11,7,7,21


In [6]:
dfall.drop(['date', 'tournament_name', 'label', 'name1', 'name2', 'field_label', 'final_score'], axis=1, inplace=True)
l = [-30239, -30238, 29800, 8072]

In [7]:
for k in range(len(dfall)):
    if dfall['field_name'][k] == 'Draw':
        dfall['winner'][k] = 999999

<ipython-input-7-5389311611b1>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfall['winner'][k] = 999999


In [8]:
dfall = dfall[~dfall.bookmaker.isin(l)]
dfall = dfall.dropna()
dfall.reset_index(inplace=True)
dfall.drop(['index'], axis=1, inplace=True)
dfall

,match,tournament_id,category,field_name,winner,odd,team1,team2,bookmaker,year,month,day_of_week,day_of_month,hour
0,25406638,178,92,Home,3346,1.08,3346.0,3343.0,30238,2021,8,2,10,17
1,28936218,679,393,Home,3052,2.45,3052.0,3245.0,-8072,2021,9,4,30,17
2,28624054,27665,49,Home,3155,1.36,3155.0,7029.0,8392,2021,9,4,30,22
3,25720692,240,165,Home,21941,3.30,21941.0,5255.0,5251,2021,10,1,4,1
4,28077158,238,44,Home,3026,2.21,3026.0,5136.0,5251,2021,9,7,26,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192950,27772134,266,66,Away,5204,3.30,5333.0,5204.0,-8072,2021,11,6,6,17
192951,27663880,211,23,Away,2406,1.56,7084.0,2406.0,6207,2021,11,6,20,14
192952,25720750,240,165,Away,241468,5.00,5257.0,241468.0,6207,2021,11,7,7,21
192953,25720750,240,165,Away,241468,4.90,5257.0,241468.0,9703,2021,11,7,7,21


In [9]:
dfall['team1'] = dfall.team1.astype(int)
dfall['team2'] = dfall.team2.astype(int)
dfall['odd'] = dfall.odd.astype(np.float32)
dfall['match'] = dfall.match.astype(int)
dfall['bookmaker'] = dfall.bookmaker.astype(int)
dfall['tournament_id'] = dfall.tournament_id.astype(int)

In [10]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(dfall['field_name'])
le.classes_

array(['Away', 'Draw', 'Home'], dtype=object)

In [11]:
dfall['field_name'] = le.transform(dfall['field_name'])

In [12]:
dfall['field_name'] = dfall.field_name.astype(int)
dfall['winner'] = dfall.winner.astype(int)

In [13]:
dfall

,match,tournament_id,category,field_name,winner,odd,team1,team2,bookmaker,year,month,day_of_week,day_of_month,hour
0,25406638,178,92,2,3346,1.08,3346,3343,30238,2021,8,2,10,17
1,28936218,679,393,2,3052,2.45,3052,3245,-8072,2021,9,4,30,17
2,28624054,27665,49,2,3155,1.36,3155,7029,8392,2021,9,4,30,22
3,25720692,240,165,2,21941,3.30,21941,5255,5251,2021,10,1,4,1
4,28077158,238,44,2,3026,2.21,3026,5136,5251,2021,9,7,26,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192950,27772134,266,66,0,5204,3.30,5333,5204,-8072,2021,11,6,6,17
192951,27663880,211,23,0,2406,1.56,7084,2406,6207,2021,11,6,20,14
192952,25720750,240,165,0,241468,5.00,5257,241468,6207,2021,11,7,7,21
192953,25720750,240,165,0,241468,4.90,5257,241468,9703,2021,11,7,7,21


In [14]:
from sklearn.model_selection import train_test_split
from sklearn import metrics

X = dfall[['match', 'tournament_id', 'category', 'field_name', 'winner', 'team1', 'team2', 'bookmaker', 'year', 'month', 'day_of_week',
       'day_of_month', 'hour']]
y = dfall['odd']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, shuffle=False)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((115773, 13), (77182, 13), (115773,), (77182,))

In [15]:
from sklearn.metrics import mean_squared_error

avg_odd = round(np.mean(y_train),2) #3.43
baseline_pred = np.repeat(avg_odd,y_test.shape[0])
baseline_rmse = np.sqrt(mean_squared_error(baseline_pred, y_test))
print("RMSE de référence sur les données de Validation :",baseline_rmse)

RMSE de référence sur les données de Validation : 2.6068196181584553


# Deep learning 

In [16]:
from tensorflow.keras import models, layers, utils, backend as K
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense

In [17]:
# create ANN model
model = Sequential()
 
# Defining the Input layer and FIRST hidden layer, both are same!
model.add(Dense(units=5, input_dim=13, kernel_initializer='normal', activation='relu'))
 
# Defining the Second layer of the model
# after the first layer we don't have to specify input_dim as keras configure it automatically
model.add(Dense(units=5, kernel_initializer='normal', activation='tanh'))


    
    
# The output neuron is a single fully connected node 
# Since we will be predicting a single number
model.add(Dense(1, kernel_initializer='normal'))
 
# Compiling the model
model.compile(loss='mean_squared_error', optimizer='adam')
 
# Fitting the ANN to the Training set
model.fit(X_train, y_train ,batch_size = 13, epochs = 10, verbose=1)

Epoch 1/10
8906/8906 [==============================] - 14s 471us/step - loss: 4.4179
Epoch 2/10
8906/8906 [==============================] - 4s 502us/step - loss: 3.4696
Epoch 3/10
8906/8906 [==============================] - 5s 546us/step - loss: 3.4040
Epoch 4/10
8906/8906 [==============================] - 5s 530us/step - loss: 3.4621
Epoch 5/10
8906/8906 [==============================] - 5s 511us/step - loss: 3.4437
Epoch 6/10
8906/8906 [==============================] - 5s 511us/step - loss: 3.5410
Epoch 7/10
8906/8906 [==============================] - 5s 517us/step - loss: 3.4813
Epoch 8/10
8906/8906 [==============================] - 4s 477us/step - loss: 3.3590
Epoch 9/10
8906/8906 [==============================] - 4s 475us/step - loss: 3.4673
Epoch 10/10
8906/8906 [==============================] - 4s 485us/step - loss: 3.4410


In [18]:
y_pred = np.round(model.predict(X_test), 2)

rmse = np.sqrt(mean_squared_error(y_pred, y_test))
train_rmse=np.sqrt(mean_squared_error(model.predict(X_train), y_train))
variance=abs(train_rmse - rmse)

print("Test RMSE for Artificial Neural Networks is ",rmse)
print("Train RMSE for Artificial Neural Networks is ",train_rmse)
print("Variance for Artificial Neural Networks is ",variance)

Test RMSE for Artificial Neural Networks is  2.616931
Train RMSE for Artificial Neural Networks is  1.8534007
Variance for Artificial Neural Networks is  0.76353025


In [19]:
# Defining a function to find the best parameters for ANN
def FunctionFindBestParams(X_train, y_train, X_test, y_test):
    
    # Defining the list of hyper parameters to try
    batch_size_list=[10, 15, 20, 50, 100, 200, 500, 1000, 2000, 5000]
    epoch_list  =   [5, 10, 15]
    
    SearchResultsData=pd.DataFrame(columns=['TrialNumber', 'Parameters', 'RMSE'])
    
    # initializing the trials
    TrialNumber=0
    for batch_size_trial in batch_size_list:
        for epochs_trial in epoch_list:
            
            TrialNumber+=1
            model = Sequential()

            model.add(Dense(units=5, input_dim=X_train.shape[1], kernel_initializer='normal', activation='relu'))
            model.add(Dense(units=5, kernel_initializer='normal', activation='relu'))
            model.add(Dense(1, kernel_initializer='normal'))
            model.compile(loss='mean_squared_error', optimizer='adam')

            model.fit(X_train, y_train ,batch_size = batch_size_trial, epochs = epochs_trial, verbose=0)
            
            RMSE = rmse = np.sqrt(mean_squared_error(np.round(model.predict(X_test), 2), y_test))
            
            print(TrialNumber, 'Parameters:','batch_size:', batch_size_trial,'-', 'epochs:',epochs_trial, 'RMSE:', RMSE)
            
            SearchResultsData=SearchResultsData.append(pd.DataFrame(data=[[TrialNumber, str(batch_size_trial)+'-'+
                                                                           str(epochs_trial), RMSE]],
                                                                    columns=['TrialNumber', 'Parameters', 'RMSE'] ))
    return(SearchResultsData)


######################################################
# Calling the function
ResultsData=FunctionFindBestParams(X_train, y_train, X_test, y_test)

1 Parameters: batch_size: 10 - epochs: 5 RMSE: 2.6092937
2 Parameters: batch_size: 10 - epochs: 10 RMSE: 2.6068196
3 Parameters: batch_size: 10 - epochs: 15 RMSE: 2.6043818
4 Parameters: batch_size: 15 - epochs: 5 RMSE: 2.6043818
5 Parameters: batch_size: 15 - epochs: 10 RMSE: 2.60198
6 Parameters: batch_size: 15 - epochs: 15 RMSE: 2.6118038
7 Parameters: batch_size: 20 - epochs: 5 RMSE: 2.884157
8 Parameters: batch_size: 20 - epochs: 10 RMSE: 2.6092937
9 Parameters: batch_size: 20 - epochs: 15 RMSE: 2.6118038
10 Parameters: batch_size: 50 - epochs: 5 RMSE: 3.7275321
11 Parameters: batch_size: 50 - epochs: 10 RMSE: 2.832599
12 Parameters: batch_size: 50 - epochs: 15 RMSE: 2.6077
13 Parameters: batch_size: 100 - epochs: 5 RMSE: 2.7587981
14 Parameters: batch_size: 100 - epochs: 10 RMSE: 2.679378
15 Parameters: batch_size: 100 - epochs: 15 RMSE: 2.6092937
16 Parameters: batch_size: 200 - epochs: 5 RMSE: 2.6705573
17 Parameters: batch_size: 200 - epochs: 10 RMSE: 3.1629415
18 Parameters: 

In [20]:
# Defining a function to find the best parameters for ANN
def FunctionFindBestParams(X_train, y_train, X_test, y_test):
    
    # Defining the list of hyper parameters to try
    batch_size_list=[10, 15, 20, 50, 100, 200, 500, 1000, 2000, 5000]
    epoch_list  =   [5, 10, 15]
    
    SearchResultsData=pd.DataFrame(columns=['TrialNumber', 'Parameters', 'RMSE'])
    
    # initializing the trials
    TrialNumber=0
    for batch_size_trial in batch_size_list:
        for epochs_trial in epoch_list:
            
            TrialNumber+=1
            model = Sequential()

            model.add(Dense(units=10, input_dim=X_train.shape[1], kernel_initializer='normal', activation='relu'))
            model.add(Dense(units=10, kernel_initializer='normal', activation='relu'))
            model.add(Dense(1, kernel_initializer='normal'))
            model.compile(loss='mean_squared_error', optimizer='adam')

            model.fit(X_train, y_train ,batch_size = batch_size_trial, epochs = epochs_trial, verbose=0)
            
            RMSE = rmse = np.sqrt(mean_squared_error(np.round(model.predict(X_test), 2), y_test))
            
            print(TrialNumber, 'Parameters:','batch_size:', batch_size_trial,'-', 'epochs:',epochs_trial, 'RMSE:', RMSE)
            
            SearchResultsData=SearchResultsData.append(pd.DataFrame(data=[[TrialNumber, str(batch_size_trial)+'-'+str(epochs_trial), RMSE]],
                                                                    columns=['TrialNumber', 'Parameters', 'RMSE'] ))
    return(SearchResultsData)


######################################################
# Calling the function
ResultsData=FunctionFindBestParams(X_train, y_train, X_test, y_test)

1 Parameters: batch_size: 10 - epochs: 5 RMSE: 2.609205
2 Parameters: batch_size: 10 - epochs: 10 RMSE: 2.6092937
3 Parameters: batch_size: 10 - epochs: 15 RMSE: 2.6092937
4 Parameters: batch_size: 15 - epochs: 5 RMSE: 2.6043818
5 Parameters: batch_size: 15 - epochs: 10 RMSE: 2.60198
6 Parameters: batch_size: 15 - epochs: 15 RMSE: 2.616931
7 Parameters: batch_size: 20 - epochs: 5 RMSE: 2.6291897
8 Parameters: batch_size: 20 - epochs: 10 RMSE: 2.6118038
9 Parameters: batch_size: 20 - epochs: 15 RMSE: 2.6043818
10 Parameters: batch_size: 50 - epochs: 5 RMSE: 2.87239
11 Parameters: batch_size: 50 - epochs: 10 RMSE: 3.6443431
12 Parameters: batch_size: 50 - epochs: 15 RMSE: 2.6068196
13 Parameters: batch_size: 100 - epochs: 5 RMSE: 2.947091
14 Parameters: batch_size: 100 - epochs: 10 RMSE: 4.010639
15 Parameters: batch_size: 100 - epochs: 15 RMSE: 3.179478
16 Parameters: batch_size: 200 - epochs: 5 RMSE: 2.6591527
17 Parameters: batch_size: 200 - epochs: 10 RMSE: 2.8447964
18 Parameters: b

In [21]:
# Defining a function to find the best parameters for ANN
def FunctionFindBestParams(X_train, y_train, X_test, y_test):
    
    # Defining the list of hyper parameters to try
    batch_size_list=[10, 15, 20, 50, 100, 200, 500, 1000, 2000, 5000]
    epoch_list  =   [5, 10, 15]
    
    SearchResultsData=pd.DataFrame(columns=['TrialNumber', 'Parameters', 'RMSE'])
    
    # initializing the trials
    TrialNumber=0
    for batch_size_trial in batch_size_list:
        for epochs_trial in epoch_list:
            
            TrialNumber+=1
            model = Sequential()

            model.add(Dense(units=20, input_dim=X_train.shape[1], kernel_initializer='normal', activation='relu'))
            model.add(Dense(units=20, kernel_initializer='normal', activation='relu'))
            model.add(Dense(1, kernel_initializer='normal'))
            model.compile(loss='mean_squared_error', optimizer='adam')

            model.fit(X_train, y_train ,batch_size = batch_size_trial, epochs = epochs_trial, verbose=0)
            
            RMSE = rmse = np.sqrt(mean_squared_error(np.round(model.predict(X_test), 2), y_test))
            
            print(TrialNumber, 'Parameters:','batch_size:', batch_size_trial,'-', 'epochs:',epochs_trial, 'RMSE:', RMSE)
            
            SearchResultsData=SearchResultsData.append(pd.DataFrame(data=[[TrialNumber, str(batch_size_trial)+'-'+str(epochs_trial), RMSE]],
                                                                    columns=['TrialNumber', 'Parameters', 'RMSE'] ))
    return(SearchResultsData)


######################################################
# Calling the function
ResultsData=FunctionFindBestParams(X_train, y_train, X_test, y_test)

1 Parameters: batch_size: 10 - epochs: 5 RMSE: 2.6092937
2 Parameters: batch_size: 10 - epochs: 10 RMSE: 2.6143494
3 Parameters: batch_size: 10 - epochs: 15 RMSE: 2.6068196
4 Parameters: batch_size: 15 - epochs: 5 RMSE: 2.5996144
5 Parameters: batch_size: 15 - epochs: 10 RMSE: 2.6043818
6 Parameters: batch_size: 15 - epochs: 15 RMSE: 2.6068196
7 Parameters: batch_size: 20 - epochs: 5 RMSE: 2.9336681
8 Parameters: batch_size: 20 - epochs: 10 RMSE: 2.6476965
9 Parameters: batch_size: 20 - epochs: 15 RMSE: 2.6068196
10 Parameters: batch_size: 50 - epochs: 5 RMSE: 5.104176
11 Parameters: batch_size: 50 - epochs: 10 RMSE: 5.9519043
12 Parameters: batch_size: 50 - epochs: 15 RMSE: 3.2274082
13 Parameters: batch_size: 100 - epochs: 5 RMSE: 7.443165
14 Parameters: batch_size: 100 - epochs: 10 RMSE: 2.8040266
15 Parameters: batch_size: 100 - epochs: 15 RMSE: 2.6961362
16 Parameters: batch_size: 200 - epochs: 5 RMSE: 3.3618612
17 Parameters: batch_size: 200 - epochs: 10 RMSE: 13.70583
18 Paramet

In [22]:
# Defining a function to find the best parameters for ANN
def FunctionFindBestParams(X_train, y_train, X_test, y_test):
    
    # Defining the list of hyper parameters to try
    batch_size_list=[10, 15, 20, 50, 100, 200, 500, 1000, 2000, 5000]
    epoch_list  =   [5, 10, 15]
    
    SearchResultsData=pd.DataFrame(columns=['TrialNumber', 'Parameters', 'RMSE'])
    
    # initializing the trials
    TrialNumber=0
    for batch_size_trial in batch_size_list:
        for epochs_trial in epoch_list:
            
            TrialNumber+=1
            model = Sequential()

            model.add(Dense(units=50, input_dim=X_train.shape[1], kernel_initializer='normal', activation='relu'))
            model.add(Dense(units=50, kernel_initializer='normal', activation='relu'))
            model.add(Dense(1, kernel_initializer='normal'))
            model.compile(loss='mean_squared_error', optimizer='adam')

            model.fit(X_train, y_train ,batch_size = batch_size_trial, epochs = epochs_trial, verbose=0)
            
            RMSE = rmse = np.sqrt(mean_squared_error(np.round(model.predict(X_test), 2), y_test))
            
            print(TrialNumber, 'Parameters:','batch_size:', batch_size_trial,'-', 'epochs:',epochs_trial, 'RMSE:', RMSE)
            
            SearchResultsData=SearchResultsData.append(pd.DataFrame(data=[[TrialNumber, str(batch_size_trial)+'-'+str(epochs_trial), RMSE]],
                                                                    columns=['TrialNumber', 'Parameters', 'RMSE'] ))
    return(SearchResultsData)


######################################################
# Calling the function
ResultsData=FunctionFindBestParams(X_train, y_train, X_test, y_test)

1 Parameters: batch_size: 10 - epochs: 5 RMSE: 2.6092937
2 Parameters: batch_size: 10 - epochs: 10 RMSE: 2.6043818
3 Parameters: batch_size: 10 - epochs: 15 RMSE: 2.6118038
4 Parameters: batch_size: 15 - epochs: 5 RMSE: 3.0844746
5 Parameters: batch_size: 15 - epochs: 10 RMSE: 2.616931
6 Parameters: batch_size: 15 - epochs: 15 RMSE: 2.60198
7 Parameters: batch_size: 20 - epochs: 5 RMSE: 16.27491
8 Parameters: batch_size: 20 - epochs: 10 RMSE: 2.6043818
9 Parameters: batch_size: 20 - epochs: 15 RMSE: 2.616931
10 Parameters: batch_size: 50 - epochs: 5 RMSE: 5.657671
11 Parameters: batch_size: 50 - epochs: 10 RMSE: 25.502047
12 Parameters: batch_size: 50 - epochs: 15 RMSE: 2.7117987
13 Parameters: batch_size: 100 - epochs: 5 RMSE: 37.07833
14 Parameters: batch_size: 100 - epochs: 10 RMSE: 7.93807
15 Parameters: batch_size: 100 - epochs: 15 RMSE: 8.030675
16 Parameters: batch_size: 200 - epochs: 5 RMSE: 8.000699
17 Parameters: batch_size: 200 - epochs: 10 RMSE: 50.904358
18 Parameters: bat

In [23]:
# Defining a function to find the best parameters for ANN
def FunctionFindBestParams(X_train, y_train, X_test, y_test):
    
    # Defining the list of hyper parameters to try
    batch_size_list=[10, 15, 20, 50, 100, 200, 500, 1000, 2000, 5000]
    epoch_list  =   [5, 10, 15]
    
    SearchResultsData=pd.DataFrame(columns=['TrialNumber', 'Parameters', 'RMSE'])
    
    # initializing the trials
    TrialNumber=0
    for batch_size_trial in batch_size_list:
        for epochs_trial in epoch_list:
            
            TrialNumber+=1
            model = Sequential()

            model.add(Dense(units=100, input_dim=X_train.shape[1], kernel_initializer='normal', activation='relu'))
            model.add(Dense(units=100, kernel_initializer='normal', activation='relu'))
            model.add(Dense(1, kernel_initializer='normal'))
            model.compile(loss='mean_squared_error', optimizer='adam')

            model.fit(X_train, y_train ,batch_size = batch_size_trial, epochs = epochs_trial, verbose=0)
            
            RMSE = rmse = np.sqrt(mean_squared_error(np.round(model.predict(X_test), 2), y_test))
            
            print(TrialNumber, 'Parameters:','batch_size:', batch_size_trial,'-', 'epochs:',epochs_trial, 'RMSE:', RMSE)
            
            SearchResultsData=SearchResultsData.append(pd.DataFrame(data=[[TrialNumber, str(batch_size_trial)+'-'+str(epochs_trial), RMSE]],
                                                                    columns=['TrialNumber', 'Parameters', 'RMSE'] ))
    return(SearchResultsData)


######################################################
# Calling the function
ResultsData=FunctionFindBestParams(X_train, y_train, X_test, y_test)

1 Parameters: batch_size: 10 - epochs: 5 RMSE: 57.95593
2 Parameters: batch_size: 10 - epochs: 10 RMSE: 145.99313
3 Parameters: batch_size: 10 - epochs: 15 RMSE: 2.6068301
4 Parameters: batch_size: 15 - epochs: 5 RMSE: 4.0697923
5 Parameters: batch_size: 15 - epochs: 10 RMSE: 2.6043818
6 Parameters: batch_size: 15 - epochs: 15 RMSE: 2.6092453
7 Parameters: batch_size: 20 - epochs: 5 RMSE: 28.432083
8 Parameters: batch_size: 20 - epochs: 10 RMSE: 2.7068367
9 Parameters: batch_size: 20 - epochs: 15 RMSE: 2.60198
10 Parameters: batch_size: 50 - epochs: 5 RMSE: 10.571062
11 Parameters: batch_size: 50 - epochs: 10 RMSE: 4.378191
12 Parameters: batch_size: 50 - epochs: 15 RMSE: 2.6395478
13 Parameters: batch_size: 100 - epochs: 5 RMSE: 30.122555
14 Parameters: batch_size: 100 - epochs: 10 RMSE: 56.523834
15 Parameters: batch_size: 100 - epochs: 15 RMSE: 3.9717174
16 Parameters: batch_size: 200 - epochs: 5 RMSE: 24.626745
17 Parameters: batch_size: 200 - epochs: 10 RMSE: 9.244997
18 Parameter

In [24]:
# Defining a function to find the best parameters for ANN
def FunctionFindBestParams(X_train, y_train, X_test, y_test):
    
    # Defining the list of hyper parameters to try
    batch_size_list=[10, 15, 20, 50, 100, 200, 500, 1000, 2000, 5000]
    epoch_list  =   [5, 10, 15]
    
    SearchResultsData=pd.DataFrame(columns=['TrialNumber', 'Parameters', 'RMSE'])
    
    # initializing the trials
    TrialNumber=0
    for batch_size_trial in batch_size_list:
        for epochs_trial in epoch_list:
            
            TrialNumber+=1
            model = Sequential()

            model.add(Dense(units=10, input_dim=X_train.shape[1], kernel_initializer='normal', activation='sigmoid'))
            model.add(Dense(units=10, kernel_initializer='normal', activation='sigmoid'))
            model.add(Dense(1, kernel_initializer='normal'))
            model.compile(loss='mean_squared_error', optimizer='adam')

            model.fit(X_train, y_train ,batch_size = batch_size_trial, epochs = epochs_trial, verbose=0)
            
            RMSE = rmse = np.sqrt(mean_squared_error(np.round(model.predict(X_test), 2), y_test))
            
            print(TrialNumber, 'Parameters:','batch_size:', batch_size_trial,'-', 'epochs:',epochs_trial, 'RMSE:', RMSE)
            
            SearchResultsData=SearchResultsData.append(pd.DataFrame(data=[[TrialNumber, str(batch_size_trial)+'-'+str(epochs_trial), RMSE]],
                                                                    columns=['TrialNumber', 'Parameters', 'RMSE'] ))
    return(SearchResultsData)


######################################################
# Calling the function
ResultsData=FunctionFindBestParams(X_train, y_train, X_test, y_test)

1 Parameters: batch_size: 10 - epochs: 5 RMSE: 2.6195478
2 Parameters: batch_size: 10 - epochs: 10 RMSE: 2.60198
3 Parameters: batch_size: 10 - epochs: 15 RMSE: 2.5904398
4 Parameters: batch_size: 15 - epochs: 5 RMSE: 2.5949924
5 Parameters: batch_size: 15 - epochs: 10 RMSE: 2.6195478
6 Parameters: batch_size: 15 - epochs: 15 RMSE: 2.627612
7 Parameters: batch_size: 20 - epochs: 5 RMSE: 2.592736
8 Parameters: batch_size: 20 - epochs: 10 RMSE: 2.6092937
9 Parameters: batch_size: 20 - epochs: 15 RMSE: 2.6143494
10 Parameters: batch_size: 50 - epochs: 5 RMSE: 2.5949924
11 Parameters: batch_size: 50 - epochs: 10 RMSE: 2.60198
12 Parameters: batch_size: 50 - epochs: 15 RMSE: 2.6195478
13 Parameters: batch_size: 100 - epochs: 5 RMSE: 2.6143494
14 Parameters: batch_size: 100 - epochs: 10 RMSE: 2.6043818
15 Parameters: batch_size: 100 - epochs: 15 RMSE: 2.5972848
16 Parameters: batch_size: 200 - epochs: 5 RMSE: 2.6068196
17 Parameters: batch_size: 200 - epochs: 10 RMSE: 2.5949924
18 Parameters

In [25]:
# Defining a function to find the best parameters for ANN
def FunctionFindBestParams(X_train, y_train, X_test, y_test):
    
    # Defining the list of hyper parameters to try
    batch_size_list=[10, 15, 20, 50, 100, 200, 500, 1000, 2000, 5000]
    epoch_list  =   [5, 10, 15]
    
    SearchResultsData=pd.DataFrame(columns=['TrialNumber', 'Parameters', 'RMSE'])
    
    # initializing the trials
    TrialNumber=0
    for batch_size_trial in batch_size_list:
        for epochs_trial in epoch_list:
            
            TrialNumber+=1
            model = Sequential()

            model.add(Dense(units=10, input_dim=X_train.shape[1], kernel_initializer='normal', activation='sigmoid'))
            model.add(Dense(units=10, kernel_initializer='normal', activation='sigmoid'))
            model.add(Dense(units=10, kernel_initializer='normal', activation='sigmoid'))
            model.add(Dense(units=10, kernel_initializer='normal', activation='sigmoid'))

            model.add(Dense(1, kernel_initializer='normal'))
            model.compile(loss='mean_squared_error', optimizer='adam')

            model.fit(X_train, y_train ,batch_size = batch_size_trial, epochs = epochs_trial, verbose=0)
            
            RMSE = rmse = np.sqrt(mean_squared_error(np.round(model.predict(X_test), 2), y_test))
            
            print(TrialNumber, 'Parameters:','batch_size:', batch_size_trial,'-', 'epochs:',epochs_trial, 'RMSE:', RMSE)
            
            SearchResultsData=SearchResultsData.append(pd.DataFrame(data=[[TrialNumber, str(batch_size_trial)+'-'+str(epochs_trial), RMSE]],
                                                                    columns=['TrialNumber', 'Parameters', 'RMSE'] ))
    return(SearchResultsData)


######################################################
# Calling the function
ResultsData=FunctionFindBestParams(X_train, y_train, X_test, y_test)

1 Parameters: batch_size: 10 - epochs: 5 RMSE: 2.6195478
2 Parameters: batch_size: 10 - epochs: 10 RMSE: 2.6043818
3 Parameters: batch_size: 10 - epochs: 15 RMSE: 2.624889
4 Parameters: batch_size: 15 - epochs: 5 RMSE: 2.6043818
5 Parameters: batch_size: 15 - epochs: 10 RMSE: 2.6331646
6 Parameters: batch_size: 15 - epochs: 15 RMSE: 2.5972848
7 Parameters: batch_size: 20 - epochs: 5 RMSE: 2.6988587
8 Parameters: batch_size: 20 - epochs: 10 RMSE: 2.6331646
9 Parameters: batch_size: 20 - epochs: 15 RMSE: 2.6092937
10 Parameters: batch_size: 50 - epochs: 5 RMSE: 2.6043818
11 Parameters: batch_size: 50 - epochs: 10 RMSE: 2.5972848
12 Parameters: batch_size: 50 - epochs: 15 RMSE: 2.6143494
13 Parameters: batch_size: 100 - epochs: 5 RMSE: 2.6195478
14 Parameters: batch_size: 100 - epochs: 10 RMSE: 2.60198
15 Parameters: batch_size: 100 - epochs: 15 RMSE: 2.6118038
16 Parameters: batch_size: 200 - epochs: 5 RMSE: 2.6092937
17 Parameters: batch_size: 200 - epochs: 10 RMSE: 2.6118038
18 Paramet

In [26]:
# Defining a function to find the best parameters for ANN
def FunctionFindBestParams(X_train, y_train, X_test, y_test):
    
    # Defining the list of hyper parameters to try
    batch_size_list=[10, 15, 20, 50, 100, 200, 500, 1000, 2000, 5000]
    epoch_list  =   [5, 10, 15]
    
    SearchResultsData=pd.DataFrame(columns=['TrialNumber', 'Parameters', 'RMSE'])
    
    # initializing the trials
    TrialNumber=0
    for batch_size_trial in batch_size_list:
        for epochs_trial in epoch_list:
            
            TrialNumber+=1
            model = Sequential()

            model.add(Dense(units=10, input_dim=X_train.shape[1], kernel_initializer='normal', activation='relu'))
            model.add(Dense(units=10, kernel_initializer='normal', activation='relu'))
            model.add(Dense(units=10, kernel_initializer='normal', activation='relu'))
            model.add(Dense(units=10, kernel_initializer='normal', activation='relu'))

            model.add(Dense(1, kernel_initializer='normal'))
            model.compile(loss='mean_squared_error', optimizer='adam')

            model.fit(X_train, y_train ,batch_size = batch_size_trial, epochs = epochs_trial, verbose=0)
            
            RMSE = rmse = np.sqrt(mean_squared_error(np.round(model.predict(X_test), 2), y_test))
            
            print(TrialNumber, 'Parameters:','batch_size:', batch_size_trial,'-', 'epochs:',epochs_trial, 'RMSE:', RMSE)
            
            SearchResultsData=SearchResultsData.append(pd.DataFrame(data=[[TrialNumber, str(batch_size_trial)+'-'+str(epochs_trial), RMSE]],
                                                                    columns=['TrialNumber', 'Parameters', 'RMSE'] ))
    return(SearchResultsData)


######################################################
# Calling the function
ResultsData=FunctionFindBestParams(X_train, y_train, X_test, y_test)

1 Parameters: batch_size: 10 - epochs: 5 RMSE: 2.7104232
2 Parameters: batch_size: 10 - epochs: 10 RMSE: 2.6775515
3 Parameters: batch_size: 10 - epochs: 15 RMSE: 2.6092937
4 Parameters: batch_size: 15 - epochs: 5 RMSE: 2.733564
5 Parameters: batch_size: 15 - epochs: 10 RMSE: 2.7073257
6 Parameters: batch_size: 15 - epochs: 15 RMSE: 2.7237356
7 Parameters: batch_size: 20 - epochs: 5 RMSE: 2.7594573
8 Parameters: batch_size: 20 - epochs: 10 RMSE: 2.6068196
9 Parameters: batch_size: 20 - epochs: 15 RMSE: 2.7400422
10 Parameters: batch_size: 50 - epochs: 5 RMSE: 2.7893991
11 Parameters: batch_size: 50 - epochs: 10 RMSE: 2.6092937
12 Parameters: batch_size: 50 - epochs: 15 RMSE: 2.6990986
13 Parameters: batch_size: 100 - epochs: 5 RMSE: 2.7685573
14 Parameters: batch_size: 100 - epochs: 10 RMSE: 2.6412332
15 Parameters: batch_size: 100 - epochs: 15 RMSE: 2.7207801
16 Parameters: batch_size: 200 - epochs: 5 RMSE: 2.6282074
17 Parameters: batch_size: 200 - epochs: 10 RMSE: 2.7072868
18 Param

In [27]:
# Defining a function to find the best parameters for ANN
def FunctionFindBestParams(X_train, y_train, X_test, y_test):
    
    # Defining the list of hyper parameters to try
    batch_size_list=[10, 15, 20, 50, 100, 200, 500, 1000, 2000, 5000]
    epoch_list  =   [5, 10, 15]
    
    SearchResultsData=pd.DataFrame(columns=['TrialNumber', 'Parameters', 'RMSE'])
    
    # initializing the trials
    TrialNumber=0
    for batch_size_trial in batch_size_list:
        for epochs_trial in epoch_list:
            
            TrialNumber+=1
            model = Sequential()

            model.add(Dense(units=10, input_dim=X_train.shape[1], kernel_initializer='normal', activation='relu'))
            model.add(Dense(units=10, kernel_initializer='normal', activation='relu'))
            model.add(Dense(units=10, kernel_initializer='normal', activation='relu'))
            model.add(Dense(units=10, kernel_initializer='normal', activation='relu'))
            model.add(Dense(units=10, kernel_initializer='normal', activation='relu'))
            model.add(Dense(units=10, kernel_initializer='normal', activation='relu'))
            model.add(Dense(units=10, kernel_initializer='normal', activation='relu'))
            model.add(Dense(units=10, kernel_initializer='normal', activation='relu'))
            model.add(Dense(units=10, kernel_initializer='normal', activation='relu'))
            model.add(Dense(units=10, kernel_initializer='normal', activation='relu'))

            model.add(Dense(1, kernel_initializer='normal'))
            model.compile(loss='mean_squared_error', optimizer='adam')

            model.fit(X_train, y_train ,batch_size = batch_size_trial, epochs = epochs_trial, verbose=0)
            
            RMSE = rmse = np.sqrt(mean_squared_error(np.round(model.predict(X_test), 2), y_test))
            
            print(TrialNumber, 'Parameters:','batch_size:', batch_size_trial,'-', 'epochs:',epochs_trial, 'RMSE:', RMSE)
            
            SearchResultsData=SearchResultsData.append(pd.DataFrame(data=[[TrialNumber, str(batch_size_trial)+'-'+str(epochs_trial), RMSE]],
                                                                    columns=['TrialNumber', 'Parameters', 'RMSE'] ))
    return(SearchResultsData)


######################################################
# Calling the function
ResultsData=FunctionFindBestParams(X_train, y_train, X_test, y_test)

1 Parameters: batch_size: 10 - epochs: 5 RMSE: 2.7143626
2 Parameters: batch_size: 10 - epochs: 10 RMSE: 2.690889
3 Parameters: batch_size: 10 - epochs: 15 RMSE: 2.6043818
4 Parameters: batch_size: 15 - epochs: 5 RMSE: 2.6405022
5 Parameters: batch_size: 15 - epochs: 10 RMSE: 2.691531
6 Parameters: batch_size: 15 - epochs: 15 RMSE: 2.592736
7 Parameters: batch_size: 20 - epochs: 5 RMSE: 2.7289762
8 Parameters: batch_size: 20 - epochs: 10 RMSE: 2.682942
9 Parameters: batch_size: 20 - epochs: 15 RMSE: 2.7442555
10 Parameters: batch_size: 50 - epochs: 5 RMSE: 2.8266833
11 Parameters: batch_size: 50 - epochs: 10 RMSE: 2.6043818
12 Parameters: batch_size: 50 - epochs: 15 RMSE: 2.7366767
13 Parameters: batch_size: 100 - epochs: 5 RMSE: 2.7088964
14 Parameters: batch_size: 100 - epochs: 10 RMSE: 2.721049
15 Parameters: batch_size: 100 - epochs: 15 RMSE: 2.7771647
16 Parameters: batch_size: 200 - epochs: 5 RMSE: 2.8217595
17 Parameters: batch_size: 200 - epochs: 10 RMSE: 2.8025756
18 Parameter

In [28]:
# Defining a function to find the best parameters for ANN
def FunctionFindBestParams(X_train, y_train, X_test, y_test):
    
    # Defining the list of hyper parameters to try
    batch_size_list=[10, 15, 20, 50, 100, 200, 500, 1000, 2000, 5000]
    epoch_list  =   [5, 10, 15]
    
    SearchResultsData=pd.DataFrame(columns=['TrialNumber', 'Parameters', 'RMSE'])
    
    # initializing the trials
    TrialNumber=0
    for batch_size_trial in batch_size_list:
        for epochs_trial in epoch_list:
            
            TrialNumber+=1
            model = Sequential()

            model.add(Dense(units=50, input_dim=X_train.shape[1], kernel_initializer='normal', activation='relu'))
            model.add(Dense(units=50, kernel_initializer='normal', activation='relu'))
            model.add(Dense(units=50, kernel_initializer='normal', activation='relu'))
            model.add(Dense(units=50, kernel_initializer='normal', activation='relu'))
            model.add(Dense(units=50, kernel_initializer='normal', activation='relu'))
            model.add(Dense(units=50, kernel_initializer='normal', activation='relu'))
            model.add(Dense(units=50, kernel_initializer='normal', activation='relu'))
            model.add(Dense(units=50, kernel_initializer='normal', activation='relu'))
            model.add(Dense(units=50, kernel_initializer='normal', activation='relu'))
            model.add(Dense(units=50, kernel_initializer='normal', activation='relu'))

            model.add(Dense(1, kernel_initializer='normal'))
            model.compile(loss='mean_squared_error', optimizer='adam')

            model.fit(X_train, y_train ,batch_size = batch_size_trial, epochs = epochs_trial, verbose=0)
            
            RMSE = rmse = np.sqrt(mean_squared_error(np.round(model.predict(X_test), 2), y_test))
            
            print(TrialNumber, 'Parameters:','batch_size:', batch_size_trial,'-', 'epochs:',epochs_trial, 'RMSE:', RMSE)
            
            SearchResultsData=SearchResultsData.append(pd.DataFrame(data=[[TrialNumber, str(batch_size_trial)+'-'+str(epochs_trial), RMSE]],
                                                                    columns=['TrialNumber', 'Parameters', 'RMSE'] ))
    return(SearchResultsData)


######################################################
# Calling the function
ResultsData=FunctionFindBestParams(X_train, y_train, X_test, y_test)

1 Parameters: batch_size: 10 - epochs: 5 RMSE: 2.5996144
2 Parameters: batch_size: 10 - epochs: 10 RMSE: 2.6417549
3 Parameters: batch_size: 10 - epochs: 15 RMSE: 2.5779736
4 Parameters: batch_size: 15 - epochs: 5 RMSE: 2.6118038
5 Parameters: batch_size: 15 - epochs: 10 RMSE: 2.7114072
6 Parameters: batch_size: 15 - epochs: 15 RMSE: 2.6303709
7 Parameters: batch_size: 20 - epochs: 5 RMSE: 2.60198
8 Parameters: batch_size: 20 - epochs: 10 RMSE: 2.7375872
9 Parameters: batch_size: 20 - epochs: 15 RMSE: 2.5949924
10 Parameters: batch_size: 50 - epochs: 5 RMSE: 2.7244148
11 Parameters: batch_size: 50 - epochs: 10 RMSE: 2.724648
12 Parameters: batch_size: 50 - epochs: 15 RMSE: 2.6068196
13 Parameters: batch_size: 100 - epochs: 5 RMSE: 2.7103853
14 Parameters: batch_size: 100 - epochs: 10 RMSE: 2.7029033
15 Parameters: batch_size: 100 - epochs: 15 RMSE: 2.6829998
16 Parameters: batch_size: 200 - epochs: 5 RMSE: 2.7427042
17 Parameters: batch_size: 200 - epochs: 10 RMSE: 2.7469392
18 Paramet

In [29]:
# Defining a function to find the best parameters for ANN
def FunctionFindBestParams(X_train, y_train, X_test, y_test):
    
    # Defining the list of hyper parameters to try
    batch_size_list=[10, 15, 20, 50, 100, 200, 500, 1000, 2000, 5000]
    epoch_list  =   [5, 10, 15]
    
    SearchResultsData=pd.DataFrame(columns=['TrialNumber', 'Parameters', 'RMSE'])
    
    # initializing the trials
    TrialNumber=0
    for batch_size_trial in batch_size_list:
        for epochs_trial in epoch_list:
            
            TrialNumber+=1
            model = Sequential()

            model.add(Dense(units=50, input_dim=X_train.shape[1], kernel_initializer='normal', activation='relu'))
            model.add(Dense(units=50, kernel_initializer='normal', activation='relu'))
            model.add(Dense(units=50, kernel_initializer='normal', activation='sigmoid'))
            model.add(Dense(units=50, kernel_initializer='normal', activation='relu'))
            model.add(Dense(units=50, kernel_initializer='normal', activation='sigmoid'))
            model.add(Dense(units=50, kernel_initializer='normal', activation='relu'))
            model.add(Dense(units=50, kernel_initializer='normal', activation='sigmoid'))
            model.add(Dense(units=50, kernel_initializer='normal', activation='relu'))
            model.add(Dense(units=50, kernel_initializer='normal', activation='sigmoid'))
            model.add(Dense(units=50, kernel_initializer='normal', activation='relu'))

            model.add(Dense(1, kernel_initializer='normal'))
            model.compile(loss='mean_squared_error', optimizer='adam')

            model.fit(X_train, y_train ,batch_size = batch_size_trial, epochs = epochs_trial, verbose=0)
            
            RMSE = rmse = np.sqrt(mean_squared_error(np.round(model.predict(X_test), 2), y_test))
            
            print(TrialNumber, 'Parameters:','batch_size:', batch_size_trial,'-', 'epochs:',epochs_trial, 'RMSE:', RMSE)
            
            SearchResultsData=SearchResultsData.append(pd.DataFrame(data=[[TrialNumber, str(batch_size_trial)+'-'+str(epochs_trial), RMSE]],
                                                                    columns=['TrialNumber', 'Parameters', 'RMSE'] ))
    return(SearchResultsData)


######################################################
# Calling the function
ResultsData=FunctionFindBestParams(X_train, y_train, X_test, y_test)

1 Parameters: batch_size: 10 - epochs: 5 RMSE: 2.5779736
2 Parameters: batch_size: 10 - epochs: 10 RMSE: 2.5799713
3 Parameters: batch_size: 10 - epochs: 15 RMSE: 2.6068196
4 Parameters: batch_size: 15 - epochs: 5 RMSE: 2.5840783
5 Parameters: batch_size: 15 - epochs: 10 RMSE: 2.6359932
6 Parameters: batch_size: 15 - epochs: 15 RMSE: 2.6043818
7 Parameters: batch_size: 20 - epochs: 5 RMSE: 2.6222007
8 Parameters: batch_size: 20 - epochs: 10 RMSE: 2.6445136
9 Parameters: batch_size: 20 - epochs: 15 RMSE: 2.627612
10 Parameters: batch_size: 50 - epochs: 5 RMSE: 2.5820062
11 Parameters: batch_size: 50 - epochs: 10 RMSE: 2.565037
12 Parameters: batch_size: 50 - epochs: 15 RMSE: 2.6331646
13 Parameters: batch_size: 100 - epochs: 5 RMSE: 2.6331646
14 Parameters: batch_size: 100 - epochs: 10 RMSE: 2.6118038
15 Parameters: batch_size: 100 - epochs: 15 RMSE: 2.6417549
16 Parameters: batch_size: 200 - epochs: 5 RMSE: 2.6043818
17 Parameters: batch_size: 200 - epochs: 10 RMSE: 2.6092937
18 Parame

In [30]:
# Defining a function to find the best parameters for ANN
def FunctionFindBestParams(X_train, y_train, X_test, y_test):
    
    # Defining the list of hyper parameters to try
    batch_size_list=[10, 15, 20, 50, 100, 200, 500, 1000, 2000, 5000]
    epoch_list  =   [5, 10, 15]
    
    SearchResultsData=pd.DataFrame(columns=['TrialNumber', 'Parameters', 'RMSE'])
    
    # initializing the trials
    TrialNumber=0
    for batch_size_trial in batch_size_list:
        for epochs_trial in epoch_list:
            
            TrialNumber+=1
            model = Sequential()

            model.add(Dense(units=50, input_dim=X_train.shape[1], kernel_initializer='normal', activation='relu'))
            model.add(Dense(units=50, kernel_initializer='normal', activation='tanh'))
            model.add(Dense(units=50, kernel_initializer='normal', activation='tanh'))
            model.add(Dense(units=50, kernel_initializer='normal', activation='tanh'))
            model.add(Dense(units=50, kernel_initializer='normal', activation='relu'))
            model.add(Dense(units=50, kernel_initializer='normal', activation='tanh'))
            model.add(Dense(units=50, kernel_initializer='normal', activation='relu'))
            model.add(Dense(units=50, kernel_initializer='normal', activation='tanh'))
            model.add(Dense(units=50, kernel_initializer='normal', activation='relu'))
            model.add(Dense(units=50, kernel_initializer='normal', activation='relu'))

            model.add(Dense(1, kernel_initializer='normal'))
            model.compile(loss='mean_squared_error', optimizer='adam')

            model.fit(X_train, y_train ,batch_size = batch_size_trial, epochs = epochs_trial, verbose=0)
            
            RMSE = rmse = np.sqrt(mean_squared_error(np.round(model.predict(X_test), 2), y_test))
            
            print(TrialNumber, 'Parameters:','batch_size:', batch_size_trial,'-', 'epochs:',epochs_trial, 'RMSE:', RMSE)
            
            SearchResultsData=SearchResultsData.append(pd.DataFrame(data=[[TrialNumber, str(batch_size_trial)+'-'+str(epochs_trial), RMSE]],
                                                                    columns=['TrialNumber', 'Parameters', 'RMSE'] ))
    return(SearchResultsData)


######################################################
# Calling the function
ResultsData=FunctionFindBestParams(X_train, y_train, X_test, y_test)

1 Parameters: batch_size: 10 - epochs: 5 RMSE: 2.6359932
2 Parameters: batch_size: 10 - epochs: 10 RMSE: 2.5972848
3 Parameters: batch_size: 10 - epochs: 15 RMSE: 2.6388566
4 Parameters: batch_size: 15 - epochs: 5 RMSE: 2.5840783
5 Parameters: batch_size: 15 - epochs: 10 RMSE: 2.6195478
6 Parameters: batch_size: 15 - epochs: 15 RMSE: 2.616931
7 Parameters: batch_size: 20 - epochs: 5 RMSE: 2.5996144
8 Parameters: batch_size: 20 - epochs: 10 RMSE: 2.616931
9 Parameters: batch_size: 20 - epochs: 15 RMSE: 2.6143494
10 Parameters: batch_size: 50 - epochs: 5 RMSE: 2.627612
11 Parameters: batch_size: 50 - epochs: 10 RMSE: 2.6043818
12 Parameters: batch_size: 50 - epochs: 15 RMSE: 2.6195478
13 Parameters: batch_size: 100 - epochs: 5 RMSE: 2.60198
14 Parameters: batch_size: 100 - epochs: 10 RMSE: 2.6068196
15 Parameters: batch_size: 100 - epochs: 15 RMSE: 2.5949924
16 Parameters: batch_size: 200 - epochs: 5 RMSE: 2.5861876
17 Parameters: batch_size: 200 - epochs: 10 RMSE: 2.5905166
18 Parameter

In [31]:
# Defining a function to find the best parameters for ANN
def FunctionFindBestParams(X_train, y_train, X_test, y_test):
    
    # Defining the list of hyper parameters to try
    batch_size_list=[10, 15, 20, 50, 100, 200, 500, 1000, 2000, 5000]
    epoch_list  =   [5, 10, 15]
    
    SearchResultsData=pd.DataFrame(columns=['TrialNumber', 'Parameters', 'RMSE'])
    
    # initializing the trials
    TrialNumber=0
    for batch_size_trial in batch_size_list:
        for epochs_trial in epoch_list:
            
            TrialNumber+=1
            model = Sequential()

            model.add(Dense(units=50, input_dim=X_train.shape[1], kernel_initializer='normal', activation='relu'))
            model.add(Dense(units=50, kernel_initializer='normal', activation='relu'))


            model.add(Dense(1, kernel_initializer='normal'))
            model.compile(loss='mean_squared_error', optimizer='adam')

            model.fit(X_train, y_train ,batch_size = batch_size_trial, epochs = epochs_trial, verbose=0)
            
            RMSE = rmse = np.sqrt(mean_squared_error(np.round(model.predict(X_test), 2), y_test))
            
            print(TrialNumber, 'Parameters:','batch_size:', batch_size_trial,'-', 'epochs:',epochs_trial, 'RMSE:', RMSE)
            
            SearchResultsData=SearchResultsData.append(pd.DataFrame(data=[[TrialNumber, str(batch_size_trial)+'-'+str(epochs_trial), RMSE]],
                                                                    columns=['TrialNumber', 'Parameters', 'RMSE'] ))
    return(SearchResultsData)


######################################################
# Calling the function
ResultsData=FunctionFindBestParams(X_train, y_train, X_test, y_test)

1 Parameters: batch_size: 10 - epochs: 5 RMSE: 2.6068196
2 Parameters: batch_size: 10 - epochs: 10 RMSE: 39.265213
3 Parameters: batch_size: 10 - epochs: 15 RMSE: 2.6068196
4 Parameters: batch_size: 15 - epochs: 5 RMSE: 2.5772195
5 Parameters: batch_size: 15 - epochs: 10 RMSE: 2.6118038
6 Parameters: batch_size: 15 - epochs: 15 RMSE: 8.3261795
7 Parameters: batch_size: 20 - epochs: 5 RMSE: 60.45394
8 Parameters: batch_size: 20 - epochs: 10 RMSE: 19.94423
9 Parameters: batch_size: 20 - epochs: 15 RMSE: 2.6043818
10 Parameters: batch_size: 50 - epochs: 5 RMSE: 43.58264
11 Parameters: batch_size: 50 - epochs: 10 RMSE: 3.010396
12 Parameters: batch_size: 50 - epochs: 15 RMSE: 17.618114
13 Parameters: batch_size: 100 - epochs: 5 RMSE: 7.0303135
14 Parameters: batch_size: 100 - epochs: 10 RMSE: 102.98632
15 Parameters: batch_size: 100 - epochs: 15 RMSE: 4.8220725
16 Parameters: batch_size: 200 - epochs: 5 RMSE: 6.715281
17 Parameters: batch_size: 200 - epochs: 10 RMSE: 13.745125
18 Parameter

In [32]:
# Defining a function to find the best parameters for ANN
def FunctionFindBestParams(X_train, y_train, X_test, y_test):
    
    # Defining the list of hyper parameters to try
    batch_size_list=[10, 15, 20, 50, 100, 200, 500, 1000, 2000, 5000]
    epoch_list  =   [5, 10, 15]
    
    SearchResultsData=pd.DataFrame(columns=['TrialNumber', 'Parameters', 'RMSE'])
    
    # initializing the trials
    TrialNumber=0
    for batch_size_trial in batch_size_list:
        for epochs_trial in epoch_list:
            TrialNumber+=1
            
            model = Sequential()
            model.add(Dense(units=4, input_dim=X_train.shape[1], kernel_initializer='normal', activation='relu'))
            model.add(Dense(units=4, kernel_initializer='normal', activation='relu'))
            model.add(Dense(1, kernel_initializer='normal'))
            
            
            model.compile(loss='mean_squared_error', optimizer='adam')

            model.fit(X_train, y_train ,batch_size = batch_size_trial, epochs = epochs_trial, verbose=0)
            
            RMSE = rmse = np.sqrt(mean_squared_error(np.round(model.predict(X_test), 2), y_test))
            
            print(TrialNumber, 'Parameters:','batch_size:', batch_size_trial,'-', 'epochs:',epochs_trial, 'RMSE:', RMSE)
            
            SearchResultsData=SearchResultsData.append(pd.DataFrame(data=[[TrialNumber, str(batch_size_trial)+'-'+str(epochs_trial), RMSE]],
                                                                    columns=['TrialNumber', 'Parameters', 'RMSE'] ))
    return(SearchResultsData)


######################################################
# Calling the function
ResultsData=FunctionFindBestParams(X_train, y_train, X_test, y_test)

1 Parameters: batch_size: 10 - epochs: 5 RMSE: 2.6043818
2 Parameters: batch_size: 10 - epochs: 10 RMSE: 2.6068196
3 Parameters: batch_size: 10 - epochs: 15 RMSE: 2.6092937
4 Parameters: batch_size: 15 - epochs: 5 RMSE: 2.6143494
5 Parameters: batch_size: 15 - epochs: 10 RMSE: 2.6068196
6 Parameters: batch_size: 15 - epochs: 15 RMSE: 2.6068196
7 Parameters: batch_size: 20 - epochs: 5 RMSE: 2.8710315
8 Parameters: batch_size: 20 - epochs: 10 RMSE: 2.6043818
9 Parameters: batch_size: 20 - epochs: 15 RMSE: 2.60198
10 Parameters: batch_size: 50 - epochs: 5 RMSE: 2.6068196
11 Parameters: batch_size: 50 - epochs: 10 RMSE: 2.8076282
12 Parameters: batch_size: 50 - epochs: 15 RMSE: 2.6118038
13 Parameters: batch_size: 100 - epochs: 5 RMSE: 2.6118038
14 Parameters: batch_size: 100 - epochs: 10 RMSE: 2.987107
15 Parameters: batch_size: 100 - epochs: 15 RMSE: 2.6068196
16 Parameters: batch_size: 200 - epochs: 5 RMSE: 2.9235594
17 Parameters: batch_size: 200 - epochs: 10 RMSE: 2.6146944
18 Paramet

In [33]:
# Defining a function to find the best parameters for ANN
def FunctionFindBestParams(X_train, y_train, X_test, y_test):
    
    # Defining the list of hyper parameters to try
    batch_size_list=[10, 15, 20, 50, 100, 200, 500, 1000, 2000, 5000]
    epoch_list  =   [5, 10, 15]
    
    SearchResultsData=pd.DataFrame(columns=['TrialNumber', 'Parameters', 'RMSE'])
    
    # initializing the trials
    TrialNumber=0
    for batch_size_trial in batch_size_list:
        for epochs_trial in epoch_list:
            TrialNumber+=1
            
            model = Sequential()
            model.add(Dense(units=3, input_dim=X_train.shape[1], kernel_initializer='normal', activation='relu'))
            model.add(Dense(units=3, kernel_initializer='normal', activation='relu'))
            model.add(Dense(1, kernel_initializer='normal'))
            
            
            model.compile(loss='mean_squared_error', optimizer='adam')

            model.fit(X_train, y_train ,batch_size = batch_size_trial, epochs = epochs_trial, verbose=0)
            
            RMSE = rmse = np.sqrt(mean_squared_error(np.round(model.predict(X_test), 2), y_test))
            
            print(TrialNumber, 'Parameters:','batch_size:', batch_size_trial,'-', 'epochs:',epochs_trial, 'RMSE:', RMSE)
            
            SearchResultsData=SearchResultsData.append(pd.DataFrame(data=[[TrialNumber, str(batch_size_trial)+'-'+str(epochs_trial), RMSE]],
                                                                    columns=['TrialNumber', 'Parameters', 'RMSE'] ))
    return(SearchResultsData)


######################################################
# Calling the function
ResultsData=FunctionFindBestParams(X_train, y_train, X_test, y_test)

1 Parameters: batch_size: 10 - epochs: 5 RMSE: 2.60198
2 Parameters: batch_size: 10 - epochs: 10 RMSE: 2.616931
3 Parameters: batch_size: 10 - epochs: 15 RMSE: 2.6092937
4 Parameters: batch_size: 15 - epochs: 5 RMSE: 2.6118038
5 Parameters: batch_size: 15 - epochs: 10 RMSE: 2.6068196
6 Parameters: batch_size: 15 - epochs: 15 RMSE: 2.6043818
7 Parameters: batch_size: 20 - epochs: 5 RMSE: 3.1859255
8 Parameters: batch_size: 20 - epochs: 10 RMSE: 2.60198
9 Parameters: batch_size: 20 - epochs: 15 RMSE: 2.6023166
10 Parameters: batch_size: 50 - epochs: 5 RMSE: 2.6991954
11 Parameters: batch_size: 50 - epochs: 10 RMSE: 2.6068196
12 Parameters: batch_size: 50 - epochs: 15 RMSE: 2.6092937
13 Parameters: batch_size: 100 - epochs: 5 RMSE: 4.423146
14 Parameters: batch_size: 100 - epochs: 10 RMSE: 2.6092937
15 Parameters: batch_size: 100 - epochs: 15 RMSE: 2.7017884
16 Parameters: batch_size: 200 - epochs: 5 RMSE: 2.9235594
17 Parameters: batch_size: 200 - epochs: 10 RMSE: 3.7132308
18 Parameters

In [34]:
# Defining a function to find the best parameters for ANN
def FunctionFindBestParams(X_train, y_train, X_test, y_test):
    
    # Defining the list of hyper parameters to try
    batch_size_list=[10, 15, 20, 50, 100, 200, 500, 1000, 2000, 5000]
    epoch_list  =   [5, 10, 15]
    
    SearchResultsData=pd.DataFrame(columns=['TrialNumber', 'Parameters', 'RMSE'])
    
    # initializing the trials
    TrialNumber=0
    for batch_size_trial in batch_size_list:
        for epochs_trial in epoch_list:
            TrialNumber+=1
            
            model = Sequential()
            model.add(Dense(units=4, input_dim=X_train.shape[1], kernel_initializer='normal', activation='relu'))
            model.add(Dense(units=4, kernel_initializer='normal', activation='relu'))
            model.add(Dense(units=4, kernel_initializer='normal', activation='relu'))

            model.add(Dense(1, kernel_initializer='normal'))
            
            
            model.compile(loss='mean_squared_error', optimizer='adam')

            model.fit(X_train, y_train ,batch_size = batch_size_trial, epochs = epochs_trial, verbose=0)
            
            RMSE = rmse = np.sqrt(mean_squared_error(np.round(model.predict(X_test), 2), y_test))
            
            print(TrialNumber, 'Parameters:','batch_size:', batch_size_trial,'-', 'epochs:',epochs_trial, 'RMSE:', RMSE)
            
            SearchResultsData=SearchResultsData.append(pd.DataFrame(data=[[TrialNumber, str(batch_size_trial)+'-'+str(epochs_trial), RMSE]],
                                                                    columns=['TrialNumber', 'Parameters', 'RMSE'] ))
    return(SearchResultsData)


######################################################
# Calling the function
ResultsData=FunctionFindBestParams(X_train, y_train, X_test, y_test)

1 Parameters: batch_size: 10 - epochs: 5 RMSE: 2.6043818
2 Parameters: batch_size: 10 - epochs: 10 RMSE: 2.5996144
3 Parameters: batch_size: 10 - epochs: 15 RMSE: 2.6118038
4 Parameters: batch_size: 15 - epochs: 5 RMSE: 2.6092937
5 Parameters: batch_size: 15 - epochs: 10 RMSE: 2.6177902
6 Parameters: batch_size: 15 - epochs: 15 RMSE: 2.6043818
7 Parameters: batch_size: 20 - epochs: 5 RMSE: 2.8235319
8 Parameters: batch_size: 20 - epochs: 10 RMSE: 2.6068196
9 Parameters: batch_size: 20 - epochs: 15 RMSE: 2.6043818
10 Parameters: batch_size: 50 - epochs: 5 RMSE: 2.6092937
11 Parameters: batch_size: 50 - epochs: 10 RMSE: 2.6092937
12 Parameters: batch_size: 50 - epochs: 15 RMSE: 2.6068196
13 Parameters: batch_size: 100 - epochs: 5 RMSE: 2.6118038
14 Parameters: batch_size: 100 - epochs: 10 RMSE: 2.6092937
15 Parameters: batch_size: 100 - epochs: 15 RMSE: 2.792496
16 Parameters: batch_size: 200 - epochs: 5 RMSE: 2.8295367
17 Parameters: batch_size: 200 - epochs: 10 RMSE: 2.6118038
18 Param

In [35]:
# Defining a function to find the best parameters for ANN
def FunctionFindBestParams(X_train, y_train, X_test, y_test):
    
    # Defining the list of hyper parameters to try
    batch_size_list=[10, 15, 20, 50, 100, 200, 500, 1000, 2000, 5000]
    epoch_list  =   [5, 10, 15]
    
    SearchResultsData=pd.DataFrame(columns=['TrialNumber', 'Parameters', 'RMSE'])
    
    # initializing the trials
    TrialNumber=0
    for batch_size_trial in batch_size_list:
        for epochs_trial in epoch_list:
            TrialNumber+=1
            
            model = Sequential()
            model.add(Dense(units=3, input_dim=X_train.shape[1], kernel_initializer='normal', activation='relu'))
            model.add(Dense(units=3, kernel_initializer='normal', activation='relu'))
            model.add(Dense(units=3, kernel_initializer='normal', activation='relu'))

            model.add(Dense(1, kernel_initializer='normal'))
            
            
            model.compile(loss='mean_squared_error', optimizer='adam')

            model.fit(X_train, y_train ,batch_size = batch_size_trial, epochs = epochs_trial, verbose=0)
            
            RMSE = rmse = np.sqrt(mean_squared_error(np.round(model.predict(X_test), 2), y_test))
            
            print(TrialNumber, 'Parameters:','batch_size:', batch_size_trial,'-', 'epochs:',epochs_trial, 'RMSE:', RMSE)
            
            SearchResultsData=SearchResultsData.append(pd.DataFrame(data=[[TrialNumber, str(batch_size_trial)+'-'+str(epochs_trial), RMSE]],
                                                                    columns=['TrialNumber', 'Parameters', 'RMSE'] ))
    return(SearchResultsData)


######################################################
# Calling the function
ResultsData=FunctionFindBestParams(X_train, y_train, X_test, y_test)

1 Parameters: batch_size: 10 - epochs: 5 RMSE: 2.6118038
2 Parameters: batch_size: 10 - epochs: 10 RMSE: 2.6043818
3 Parameters: batch_size: 10 - epochs: 15 RMSE: 2.6922183
4 Parameters: batch_size: 15 - epochs: 5 RMSE: 2.7116852
5 Parameters: batch_size: 15 - epochs: 10 RMSE: 2.6068196
6 Parameters: batch_size: 15 - epochs: 15 RMSE: 2.69889
7 Parameters: batch_size: 20 - epochs: 5 RMSE: 2.6092937
8 Parameters: batch_size: 20 - epochs: 10 RMSE: 2.708773
9 Parameters: batch_size: 20 - epochs: 15 RMSE: 2.7159839
10 Parameters: batch_size: 50 - epochs: 5 RMSE: 2.6618316
11 Parameters: batch_size: 50 - epochs: 10 RMSE: 2.670584
12 Parameters: batch_size: 50 - epochs: 15 RMSE: 2.6043818
13 Parameters: batch_size: 100 - epochs: 5 RMSE: 2.740384
14 Parameters: batch_size: 100 - epochs: 10 RMSE: 2.6092937
15 Parameters: batch_size: 100 - epochs: 15 RMSE: 2.6118038
16 Parameters: batch_size: 200 - epochs: 5 RMSE: 2.9235594
17 Parameters: batch_size: 200 - epochs: 10 RMSE: 2.6118038
18 Parameter

In [36]:
# Defining a function to find the best parameters for ANN
def FunctionFindBestParams(X_train, y_train, X_test, y_test):
    
    # Defining the list of hyper parameters to try
    batch_size_list=[10, 15, 20, 50, 100, 200, 500, 1000, 2000, 5000]
    epoch_list  =   [5, 10, 15]
    
    SearchResultsData=pd.DataFrame(columns=['TrialNumber', 'Parameters', 'RMSE'])
    
    # initializing the trials
    TrialNumber=0
    for batch_size_trial in batch_size_list:
        for epochs_trial in epoch_list:
            TrialNumber+=1
            
            model = Sequential()
            model.add(Dense(units=2, input_dim=X_train.shape[1], kernel_initializer='normal', activation='relu'))
            model.add(Dense(units=2, kernel_initializer='normal', activation='relu'))
            model.add(Dense(units=2, kernel_initializer='normal', activation='relu'))

            model.add(Dense(1, kernel_initializer='normal'))
            
            
            model.compile(loss='mean_squared_error', optimizer='adam')

            model.fit(X_train, y_train ,batch_size = batch_size_trial, epochs = epochs_trial, verbose=0)
            
            RMSE = np.sqrt(mean_squared_error(np.round(model.predict(X_test), 2), y_test))
            
            print(TrialNumber, 'Parameters:','batch_size:', batch_size_trial,'-', 'epochs:',epochs_trial, 'RMSE:', RMSE)
            
            SearchResultsData=SearchResultsData.append(pd.DataFrame(data=[[TrialNumber, str(batch_size_trial)+'-'+str(epochs_trial), RMSE]],
                                                                    columns=['TrialNumber', 'Parameters', 'RMSE'] ))
    return(SearchResultsData)


######################################################
# Calling the function
ResultsData=FunctionFindBestParams(X_train, y_train, X_test, y_test)

1 Parameters: batch_size: 10 - epochs: 5 RMSE: 2.6118038
2 Parameters: batch_size: 10 - epochs: 10 RMSE: 2.6143494
3 Parameters: batch_size: 10 - epochs: 15 RMSE: 2.6092937
4 Parameters: batch_size: 15 - epochs: 5 RMSE: 2.6092937
5 Parameters: batch_size: 15 - epochs: 10 RMSE: 2.7139966
6 Parameters: batch_size: 15 - epochs: 15 RMSE: 2.60198
7 Parameters: batch_size: 20 - epochs: 5 RMSE: 2.7681537
8 Parameters: batch_size: 20 - epochs: 10 RMSE: 2.6092937
9 Parameters: batch_size: 20 - epochs: 15 RMSE: 2.6068196
10 Parameters: batch_size: 50 - epochs: 5 RMSE: 2.6092937
11 Parameters: batch_size: 50 - epochs: 10 RMSE: 2.6118038
12 Parameters: batch_size: 50 - epochs: 15 RMSE: 2.8143687
13 Parameters: batch_size: 100 - epochs: 5 RMSE: 2.6848888
14 Parameters: batch_size: 100 - epochs: 10 RMSE: 2.6068196
15 Parameters: batch_size: 100 - epochs: 15 RMSE: 2.6092937
16 Parameters: batch_size: 200 - epochs: 5 RMSE: 2.9235594
17 Parameters: batch_size: 200 - epochs: 10 RMSE: 2.6118038
18 Parame

In [37]:
# Defining a function to find the best parameters for ANN
def FunctionFindBestParams(X_train, y_train, X_test, y_test):
    
    # Defining the list of hyper parameters to try
    batch_size_list=[10, 15, 20, 50, 100, 200, 500, 1000, 2000, 5000]
    epoch_list  =   [5, 10, 15]
    
    SearchResultsData=pd.DataFrame(columns=['TrialNumber', 'Parameters', 'RMSE'])
    
    # initializing the trials
    TrialNumber=0
    for batch_size_trial in batch_size_list:
        for epochs_trial in epoch_list:
            TrialNumber+=1
            
            model = Sequential()
            model.add(Dense(units=5, input_dim=X_train.shape[1], kernel_initializer='normal', activation='relu'))
            model.add(Dense(units=5, kernel_initializer='normal', activation='relu'))
            model.add(Dense(units=5, kernel_initializer='normal', activation='relu'))

            model.add(Dense(1, kernel_initializer='normal'))
            
            
            model.compile(loss='mean_squared_error', optimizer='adam')

            model.fit(X_train, y_train ,batch_size = batch_size_trial, epochs = epochs_trial, verbose=0)
            
            RMSE = np.sqrt(mean_squared_error(np.round(model.predict(X_test), 2), y_test))
            
            print(TrialNumber, 'Parameters:','batch_size:', batch_size_trial,'-', 'epochs:',epochs_trial, 'RMSE:', RMSE)
            
            SearchResultsData=SearchResultsData.append(pd.DataFrame(data=[[TrialNumber, str(batch_size_trial)+'-'+str(epochs_trial), RMSE]],
                                                                    columns=['TrialNumber', 'Parameters', 'RMSE'] ))
    return(SearchResultsData)


######################################################
# Calling the function
ResultsData=FunctionFindBestParams(X_train, y_train, X_test, y_test)

1 Parameters: batch_size: 10 - epochs: 5 RMSE: 2.6092937
2 Parameters: batch_size: 10 - epochs: 10 RMSE: 2.6303709
3 Parameters: batch_size: 10 - epochs: 15 RMSE: 2.699656
4 Parameters: batch_size: 15 - epochs: 5 RMSE: 2.660985
5 Parameters: batch_size: 15 - epochs: 10 RMSE: 2.6118038
6 Parameters: batch_size: 15 - epochs: 15 RMSE: 2.6043818
7 Parameters: batch_size: 20 - epochs: 5 RMSE: 2.659981
8 Parameters: batch_size: 20 - epochs: 10 RMSE: 2.7086866
9 Parameters: batch_size: 20 - epochs: 15 RMSE: 2.6092937
10 Parameters: batch_size: 50 - epochs: 5 RMSE: 2.6092937
11 Parameters: batch_size: 50 - epochs: 10 RMSE: 2.6092937
12 Parameters: batch_size: 50 - epochs: 15 RMSE: 2.680626
13 Parameters: batch_size: 100 - epochs: 5 RMSE: 4.12058
14 Parameters: batch_size: 100 - epochs: 10 RMSE: 2.6783974
15 Parameters: batch_size: 100 - epochs: 15 RMSE: 2.872504
16 Parameters: batch_size: 200 - epochs: 5 RMSE: 2.5679414
17 Parameters: batch_size: 200 - epochs: 10 RMSE: 2.7391994
18 Parameters:

In [38]:
# Defining a function to find the best parameters for ANN
def FunctionFindBestParams(X_train, y_train, X_test, y_test):
    
    # Defining the list of hyper parameters to try
    batch_size_list=[10, 15, 20, 50, 100, 200, 500, 1000, 2000, 5000]
    epoch_list  =   [5, 10, 15]
    
    SearchResultsData=pd.DataFrame(columns=['TrialNumber', 'Parameters', 'RMSE'])
    
    # initializing the trials
    TrialNumber=0
    for batch_size_trial in batch_size_list:
        for epochs_trial in epoch_list:
            TrialNumber+=1
            
            model = Sequential()
            model.add(Dense(units=6, input_dim=X_train.shape[1], kernel_initializer='normal', activation='relu'))
            model.add(Dense(units=6, kernel_initializer='normal', activation='relu'))
            model.add(Dense(units=6, kernel_initializer='normal', activation='relu'))

            model.add(Dense(1, kernel_initializer='normal'))
            
            
            model.compile(loss='mean_squared_error', optimizer='adam')

            model.fit(X_train, y_train ,batch_size = batch_size_trial, epochs = epochs_trial, verbose=0)
            
            RMSE = np.sqrt(mean_squared_error(np.round(model.predict(X_test), 2), y_test))
            
            print(TrialNumber, 'Parameters:','batch_size:', batch_size_trial,'-', 'epochs:',epochs_trial, 'RMSE:', RMSE)
            
            SearchResultsData=SearchResultsData.append(pd.DataFrame(data=[[TrialNumber, str(batch_size_trial)+'-'+str(epochs_trial), RMSE]],
                                                                    columns=['TrialNumber', 'Parameters', 'RMSE'] ))
    return(SearchResultsData)


######################################################
# Calling the function
ResultsData=FunctionFindBestParams(X_train, y_train, X_test, y_test)

1 Parameters: batch_size: 10 - epochs: 5 RMSE: 2.6068196
2 Parameters: batch_size: 10 - epochs: 10 RMSE: 2.7220738
3 Parameters: batch_size: 10 - epochs: 15 RMSE: 2.6957936
4 Parameters: batch_size: 15 - epochs: 5 RMSE: 2.8082032
5 Parameters: batch_size: 15 - epochs: 10 RMSE: 2.6143494
6 Parameters: batch_size: 15 - epochs: 15 RMSE: 2.7004511
7 Parameters: batch_size: 20 - epochs: 5 RMSE: 2.844577
8 Parameters: batch_size: 20 - epochs: 10 RMSE: 2.7262573
9 Parameters: batch_size: 20 - epochs: 15 RMSE: 2.6994972
10 Parameters: batch_size: 50 - epochs: 5 RMSE: 2.7909398
11 Parameters: batch_size: 50 - epochs: 10 RMSE: 2.6092937
12 Parameters: batch_size: 50 - epochs: 15 RMSE: 2.6816442
13 Parameters: batch_size: 100 - epochs: 5 RMSE: 2.6142433
14 Parameters: batch_size: 100 - epochs: 10 RMSE: 2.7719464
15 Parameters: batch_size: 100 - epochs: 15 RMSE: 2.6092937
16 Parameters: batch_size: 200 - epochs: 5 RMSE: 3.9854968
17 Parameters: batch_size: 200 - epochs: 10 RMSE: 2.6143494
18 Param

In [39]:
# Defining a function to find the best parameters for ANN
def FunctionFindBestParams(X_train, y_train, X_test, y_test):
    
    # Defining the list of hyper parameters to try
    batch_size_list=[10, 15, 20, 50, 100, 200, 500, 1000, 2000, 5000]
    epoch_list  =   [5, 10, 15]
    
    SearchResultsData=pd.DataFrame(columns=['TrialNumber', 'Parameters', 'RMSE'])
    
    # initializing the trials
    TrialNumber=0
    for batch_size_trial in batch_size_list:
        for epochs_trial in epoch_list:
            TrialNumber+=1
            
            model = Sequential()
            model.add(Dense(units=7, input_dim=X_train.shape[1], kernel_initializer='normal', activation='relu'))
            model.add(Dense(units=7, kernel_initializer='normal', activation='relu'))
            model.add(Dense(units=7, kernel_initializer='normal', activation='relu'))

            model.add(Dense(1, kernel_initializer='normal'))
            
            
            model.compile(loss='mean_squared_error', optimizer='adam')

            model.fit(X_train, y_train ,batch_size = batch_size_trial, epochs = epochs_trial, verbose=0)
            
            RMSE = np.sqrt(mean_squared_error(np.round(model.predict(X_test), 2), y_test))
            
            print(TrialNumber, 'Parameters:','batch_size:', batch_size_trial,'-', 'epochs:',epochs_trial, 'RMSE:', RMSE)
            
            SearchResultsData=SearchResultsData.append(pd.DataFrame(data=[[TrialNumber, str(batch_size_trial)+'-'+str(epochs_trial), RMSE]],
                                                                    columns=['TrialNumber', 'Parameters', 'RMSE'] ))
    return(SearchResultsData)


######################################################
# Calling the function
ResultsData=FunctionFindBestParams(X_train, y_train, X_test, y_test)

1 Parameters: batch_size: 10 - epochs: 5 RMSE: 2.6092937
2 Parameters: batch_size: 10 - epochs: 10 RMSE: 2.7079284
3 Parameters: batch_size: 10 - epochs: 15 RMSE: 2.6092937
4 Parameters: batch_size: 15 - epochs: 5 RMSE: 2.6092937
5 Parameters: batch_size: 15 - epochs: 10 RMSE: 2.7081437
6 Parameters: batch_size: 15 - epochs: 15 RMSE: 2.69518
7 Parameters: batch_size: 20 - epochs: 5 RMSE: 2.6556385
8 Parameters: batch_size: 20 - epochs: 10 RMSE: 2.704935
9 Parameters: batch_size: 20 - epochs: 15 RMSE: 2.6043818
10 Parameters: batch_size: 50 - epochs: 5 RMSE: 2.6068196
11 Parameters: batch_size: 50 - epochs: 10 RMSE: 2.7269554
12 Parameters: batch_size: 50 - epochs: 15 RMSE: 2.6043818
13 Parameters: batch_size: 100 - epochs: 5 RMSE: 3.9332347
14 Parameters: batch_size: 100 - epochs: 10 RMSE: 2.7556186
15 Parameters: batch_size: 100 - epochs: 15 RMSE: 2.7945106
16 Parameters: batch_size: 200 - epochs: 5 RMSE: 2.7987404
17 Parameters: batch_size: 200 - epochs: 10 RMSE: 2.6274068
18 Paramet

In [40]:
# Defining a function to find the best parameters for ANN
def FunctionFindBestParams(X_train, y_train, X_test, y_test):
    
    # Defining the list of hyper parameters to try
    batch_size_list=[10, 15, 20, 50, 100, 200, 500, 1000, 2000, 5000]
    epoch_list  =   [5, 10, 15]
    
    SearchResultsData=pd.DataFrame(columns=['TrialNumber', 'Parameters', 'RMSE'])
    
    # initializing the trials
    TrialNumber=0
    for batch_size_trial in batch_size_list:
        for epochs_trial in epoch_list:
            TrialNumber+=1
            
            model = Sequential()
            model.add(Dense(units=8, input_dim=X_train.shape[1], kernel_initializer='normal', activation='relu'))
            model.add(Dense(units=8, kernel_initializer='normal', activation='relu'))
            model.add(Dense(units=8, kernel_initializer='normal', activation='relu'))

            model.add(Dense(1, kernel_initializer='normal'))
            
            
            model.compile(loss='mean_squared_error', optimizer='adam')

            model.fit(X_train, y_train ,batch_size = batch_size_trial, epochs = epochs_trial, verbose=0)
            
            RMSE = np.sqrt(mean_squared_error(np.round(model.predict(X_test), 2), y_test))
            
            print(TrialNumber, 'Parameters:','batch_size:', batch_size_trial,'-', 'epochs:',epochs_trial, 'RMSE:', RMSE)
            
            SearchResultsData=SearchResultsData.append(pd.DataFrame(data=[[TrialNumber, str(batch_size_trial)+'-'+str(epochs_trial), RMSE]],
                                                                    columns=['TrialNumber', 'Parameters', 'RMSE'] ))
    return(SearchResultsData)


######################################################
# Calling the function
ResultsData=FunctionFindBestParams(X_train, y_train, X_test, y_test)

1 Parameters: batch_size: 10 - epochs: 5 RMSE: 2.6118038
2 Parameters: batch_size: 10 - epochs: 10 RMSE: 2.6973825
3 Parameters: batch_size: 10 - epochs: 15 RMSE: 2.7038975
4 Parameters: batch_size: 15 - epochs: 5 RMSE: 2.6043818
5 Parameters: batch_size: 15 - epochs: 10 RMSE: 2.5995853
6 Parameters: batch_size: 15 - epochs: 15 RMSE: 2.5996144
7 Parameters: batch_size: 20 - epochs: 5 RMSE: 2.606798
8 Parameters: batch_size: 20 - epochs: 10 RMSE: 2.6118038
9 Parameters: batch_size: 20 - epochs: 15 RMSE: 2.711318
10 Parameters: batch_size: 50 - epochs: 5 RMSE: 2.9233618
11 Parameters: batch_size: 50 - epochs: 10 RMSE: 2.7744472
12 Parameters: batch_size: 50 - epochs: 15 RMSE: 2.6169994
13 Parameters: batch_size: 100 - epochs: 5 RMSE: 2.6924765
14 Parameters: batch_size: 100 - epochs: 10 RMSE: 2.808275
15 Parameters: batch_size: 100 - epochs: 15 RMSE: 2.7154927
16 Parameters: batch_size: 200 - epochs: 5 RMSE: 3.2077472
17 Parameters: batch_size: 200 - epochs: 10 RMSE: 2.7296631
18 Paramet

In [41]:
# Defining a function to find the best parameters for ANN
def FunctionFindBestParams(X_train, y_train, X_test, y_test):
    
    # Defining the list of hyper parameters to try
    batch_size_list=[10, 15, 20, 50, 100, 200, 500, 1000, 2000, 5000]
    epoch_list  =   [5, 10, 15]
    
    SearchResultsData=pd.DataFrame(columns=['TrialNumber', 'Parameters', 'RMSE'])
    
    # initializing the trials
    TrialNumber=0
    for batch_size_trial in batch_size_list:
        for epochs_trial in epoch_list:
            TrialNumber+=1
            
            model = Sequential()
            model.add(Dense(units=9, input_dim=X_train.shape[1], kernel_initializer='normal', activation='relu'))
            model.add(Dense(units=9, kernel_initializer='normal', activation='relu'))
            model.add(Dense(units=9, kernel_initializer='normal', activation='relu'))

            model.add(Dense(1, kernel_initializer='normal'))
            
            
            model.compile(loss='mean_squared_error', optimizer='adam')

            model.fit(X_train, y_train ,batch_size = batch_size_trial, epochs = epochs_trial, verbose=0)
            
            RMSE = np.sqrt(mean_squared_error(np.round(model.predict(X_test), 2), y_test))
            
            print(TrialNumber, 'Parameters:','batch_size:', batch_size_trial,'-', 'epochs:',epochs_trial, 'RMSE:', RMSE)
            
            SearchResultsData=SearchResultsData.append(pd.DataFrame(data=[[TrialNumber, str(batch_size_trial)+'-'+str(epochs_trial), RMSE]],
                                                                    columns=['TrialNumber', 'Parameters', 'RMSE'] ))
    return(SearchResultsData)


######################################################
# Calling the function
ResultsData=FunctionFindBestParams(X_train, y_train, X_test, y_test)

1 Parameters: batch_size: 10 - epochs: 5 RMSE: 2.6143494
2 Parameters: batch_size: 10 - epochs: 10 RMSE: 2.6941214
3 Parameters: batch_size: 10 - epochs: 15 RMSE: 2.6143494
4 Parameters: batch_size: 15 - epochs: 5 RMSE: 2.6092937
5 Parameters: batch_size: 15 - epochs: 10 RMSE: 2.7120776
6 Parameters: batch_size: 15 - epochs: 15 RMSE: 2.60198
7 Parameters: batch_size: 20 - epochs: 5 RMSE: 2.692076
8 Parameters: batch_size: 20 - epochs: 10 RMSE: 2.6092937
9 Parameters: batch_size: 20 - epochs: 15 RMSE: 2.7058837
10 Parameters: batch_size: 50 - epochs: 5 RMSE: 2.7168508
11 Parameters: batch_size: 50 - epochs: 10 RMSE: 2.7143307
12 Parameters: batch_size: 50 - epochs: 15 RMSE: 2.7316198
13 Parameters: batch_size: 100 - epochs: 5 RMSE: 5.2002883
14 Parameters: batch_size: 100 - epochs: 10 RMSE: 2.58305
15 Parameters: batch_size: 100 - epochs: 15 RMSE: 2.6054895
16 Parameters: batch_size: 200 - epochs: 5 RMSE: 2.8785644
17 Parameters: batch_size: 200 - epochs: 10 RMSE: 3.420324
18 Parameters

In [42]:
# Defining a function to find the best parameters for ANN
def FunctionFindBestParams(X_train, y_train, X_test, y_test):
    
    # Defining the list of hyper parameters to try
    batch_size_list=[10, 15, 20, 50, 100, 200, 500, 1000, 2000, 5000]
    epoch_list  =   [5, 10, 15]
    
    SearchResultsData=pd.DataFrame(columns=['TrialNumber', 'Parameters', 'RMSE'])
    
    # initializing the trials
    TrialNumber=0
    for batch_size_trial in batch_size_list:
        for epochs_trial in epoch_list:
            TrialNumber+=1
            
            model = Sequential()
            model.add(Dense(units=5, input_dim=X_train.shape[1], kernel_initializer='normal', activation='sigmoid'))
            model.add(Dense(units=5, kernel_initializer='normal', activation='sigmoid'))
            model.add(Dense(units=5, kernel_initializer='normal', activation='sigmoid'))

            model.add(Dense(1, kernel_initializer='normal'))
            
            
            model.compile(loss='mean_squared_error', optimizer='adam')

            model.fit(X_train, y_train ,batch_size = batch_size_trial, epochs = epochs_trial, verbose=0)
            
            RMSE = np.sqrt(mean_squared_error(np.round(model.predict(X_test), 2), y_test))
            
            print(TrialNumber, 'Parameters:','batch_size:', batch_size_trial,'-', 'epochs:',epochs_trial, 'RMSE:', RMSE)
            
            SearchResultsData=SearchResultsData.append(pd.DataFrame(data=[[TrialNumber, str(batch_size_trial)+'-'+str(epochs_trial), RMSE]],
                                                                    columns=['TrialNumber', 'Parameters', 'RMSE'] ))
    return(SearchResultsData)


######################################################
# Calling the function
ResultsData=FunctionFindBestParams(X_train, y_train, X_test, y_test)

1 Parameters: batch_size: 10 - epochs: 5 RMSE: 2.6331646
2 Parameters: batch_size: 10 - epochs: 10 RMSE: 2.592736
3 Parameters: batch_size: 10 - epochs: 15 RMSE: 2.5996144
4 Parameters: batch_size: 15 - epochs: 5 RMSE: 2.616931
5 Parameters: batch_size: 15 - epochs: 10 RMSE: 2.6143494
6 Parameters: batch_size: 15 - epochs: 15 RMSE: 2.6195478
7 Parameters: batch_size: 20 - epochs: 5 RMSE: 2.6068196
8 Parameters: batch_size: 20 - epochs: 10 RMSE: 2.6092937
9 Parameters: batch_size: 20 - epochs: 15 RMSE: 2.5996144
10 Parameters: batch_size: 50 - epochs: 5 RMSE: 2.6068196
11 Parameters: batch_size: 50 - epochs: 10 RMSE: 2.5996144
12 Parameters: batch_size: 50 - epochs: 15 RMSE: 2.6092937
13 Parameters: batch_size: 100 - epochs: 5 RMSE: 2.6092937
14 Parameters: batch_size: 100 - epochs: 10 RMSE: 2.6068196
15 Parameters: batch_size: 100 - epochs: 15 RMSE: 2.6092937
16 Parameters: batch_size: 200 - epochs: 5 RMSE: 2.6092937
17 Parameters: batch_size: 200 - epochs: 10 RMSE: 2.5996144
18 Parame

In [43]:
# Defining a function to find the best parameters for ANN
def FunctionFindBestParams(X_train, y_train, X_test, y_test):
    
    # Defining the list of hyper parameters to try
    batch_size_list=[10, 15, 20, 50, 100, 200, 500, 1000, 2000, 5000]
    epoch_list  =   [5, 10, 15]
    
    SearchResultsData=pd.DataFrame(columns=['TrialNumber', 'Parameters', 'RMSE'])
    
    # initializing the trials
    TrialNumber=0
    for batch_size_trial in batch_size_list:
        for epochs_trial in epoch_list:
            TrialNumber+=1
            
            model = Sequential()
            model.add(Dense(units=7, input_dim=X_train.shape[1], kernel_initializer='normal', activation='sigmoid'))
            model.add(Dense(units=7, kernel_initializer='normal', activation='sigmoid'))
            model.add(Dense(units=7, kernel_initializer='normal', activation='sigmoid'))

            model.add(Dense(1, kernel_initializer='normal'))
            
            
            model.compile(loss='mean_squared_error', optimizer='adam')

            model.fit(X_train, y_train ,batch_size = batch_size_trial, epochs = epochs_trial, verbose=0)
            
            RMSE = np.sqrt(mean_squared_error(np.round(model.predict(X_test), 2), y_test))
            
            print(TrialNumber, 'Parameters:','batch_size:', batch_size_trial,'-', 'epochs:',epochs_trial, 'RMSE:', RMSE)
            
            SearchResultsData=SearchResultsData.append(pd.DataFrame(data=[[TrialNumber, str(batch_size_trial)+'-'+str(epochs_trial), RMSE]],
                                                                    columns=['TrialNumber', 'Parameters', 'RMSE'] ))
    return(SearchResultsData)


######################################################
# Calling the function
ResultsData=FunctionFindBestParams(X_train, y_train, X_test, y_test)

1 Parameters: batch_size: 10 - epochs: 5 RMSE: 2.5972848
2 Parameters: batch_size: 10 - epochs: 10 RMSE: 2.6092937
3 Parameters: batch_size: 10 - epochs: 15 RMSE: 2.6331646
4 Parameters: batch_size: 15 - epochs: 5 RMSE: 2.5905166
5 Parameters: batch_size: 15 - epochs: 10 RMSE: 2.6143494
6 Parameters: batch_size: 15 - epochs: 15 RMSE: 2.7137208
7 Parameters: batch_size: 20 - epochs: 5 RMSE: 2.5972848
8 Parameters: batch_size: 20 - epochs: 10 RMSE: 2.6143494
9 Parameters: batch_size: 20 - epochs: 15 RMSE: 2.6092937
10 Parameters: batch_size: 50 - epochs: 5 RMSE: 2.6068196
11 Parameters: batch_size: 50 - epochs: 10 RMSE: 2.6092937
12 Parameters: batch_size: 50 - epochs: 15 RMSE: 2.6092937
13 Parameters: batch_size: 100 - epochs: 5 RMSE: 2.6195478
14 Parameters: batch_size: 100 - epochs: 10 RMSE: 2.6092937
15 Parameters: batch_size: 100 - epochs: 15 RMSE: 2.6068196
16 Parameters: batch_size: 200 - epochs: 5 RMSE: 2.6043818
17 Parameters: batch_size: 200 - epochs: 10 RMSE: 2.6043818
18 Para

In [44]:
# Defining a function to find the best parameters for ANN
def FunctionFindBestParams(X_train, y_train, X_test, y_test):
    
    # Defining the list of hyper parameters to try
    batch_size_list=[10, 15, 20, 50, 100, 200, 500, 1000, 2000, 5000]
    epoch_list  =   [5, 10, 15]
    
    SearchResultsData=pd.DataFrame(columns=['TrialNumber', 'Parameters', 'RMSE'])
    
    # initializing the trials
    TrialNumber=0
    for batch_size_trial in batch_size_list:
        for epochs_trial in epoch_list:
            TrialNumber+=1
            
            model = Sequential()
            model.add(Dense(units=7, input_dim=X_train.shape[1], kernel_initializer='normal', activation='sigmoid'))
            model.add(Dense(units=7, kernel_initializer='normal', activation='sigmoid'))
            model.add(Dense(units=7, kernel_initializer='normal', activation='sigmoid'))

            model.add(Dense(1, kernel_initializer='normal'))
            
            
            model.compile(loss='mse', optimizer='adam')

            model.fit(X_train, y_train ,batch_size = batch_size_trial, epochs = epochs_trial, verbose=0)
            
            RMSE = np.sqrt(mean_squared_error(np.round(model.predict(X_test), 2), y_test))
            
            print(TrialNumber, 'Parameters:','batch_size:', batch_size_trial,'-', 'epochs:',epochs_trial, 'RMSE:', RMSE)
            
            SearchResultsData=SearchResultsData.append(pd.DataFrame(data=[[TrialNumber, str(batch_size_trial)+'-'+str(epochs_trial), RMSE]],
                                                                    columns=['TrialNumber', 'Parameters', 'RMSE'] ))
    return(SearchResultsData)


######################################################
# Calling the function
ResultsData=FunctionFindBestParams(X_train, y_train, X_test, y_test)

1 Parameters: batch_size: 10 - epochs: 5 RMSE: 2.5905166
2 Parameters: batch_size: 10 - epochs: 10 RMSE: 2.5972848
3 Parameters: batch_size: 10 - epochs: 15 RMSE: 2.6143494
4 Parameters: batch_size: 15 - epochs: 5 RMSE: 2.6092842
5 Parameters: batch_size: 15 - epochs: 10 RMSE: 2.5883336
6 Parameters: batch_size: 15 - epochs: 15 RMSE: 2.60198
7 Parameters: batch_size: 20 - epochs: 5 RMSE: 2.6068196
8 Parameters: batch_size: 20 - epochs: 10 RMSE: 2.616931
9 Parameters: batch_size: 20 - epochs: 15 RMSE: 2.5972848
10 Parameters: batch_size: 50 - epochs: 5 RMSE: 2.6043818
11 Parameters: batch_size: 50 - epochs: 10 RMSE: 2.60198
12 Parameters: batch_size: 50 - epochs: 15 RMSE: 2.6068196
13 Parameters: batch_size: 100 - epochs: 5 RMSE: 2.6043818
14 Parameters: batch_size: 100 - epochs: 10 RMSE: 2.6092937
15 Parameters: batch_size: 100 - epochs: 15 RMSE: 2.6118038
16 Parameters: batch_size: 200 - epochs: 5 RMSE: 2.6092937
17 Parameters: batch_size: 200 - epochs: 10 RMSE: 2.6068196
18 Parameter

In [45]:
# Defining a function to find the best parameters for ANN
def FunctionFindBestParams(X_train, y_train, X_test, y_test):
    
    # Defining the list of hyper parameters to try
    batch_size_list=[10, 15, 20, 50, 100, 200, 500, 1000, 2000, 5000]
    epoch_list  =   [5, 10, 15]
    
    SearchResultsData=pd.DataFrame(columns=['TrialNumber', 'Parameters', 'RMSE'])
    
    # initializing the trials
    TrialNumber=0
    for batch_size_trial in batch_size_list:
        for epochs_trial in epoch_list:
            TrialNumber+=1
            
            model = Sequential()
            model.add(Dense(units=2, input_dim=X_train.shape[1], kernel_initializer='normal', activation='sigmoid'))
            model.add(Dense(units=2, kernel_initializer='normal', activation='sigmoid'))
            model.add(Dense(units=2, kernel_initializer='normal', activation='sigmoid'))

            model.add(Dense(1, kernel_initializer='normal'))
            
            
            model.compile(loss='mse', optimizer='adam')

            model.fit(X_train, y_train ,batch_size = batch_size_trial, epochs = epochs_trial, verbose=0)
            
            RMSE = np.sqrt(mean_squared_error(np.round(model.predict(X_test), 2), y_test))
            
            print(TrialNumber, 'Parameters:','batch_size:', batch_size_trial,'-', 'epochs:',epochs_trial, 'RMSE:', RMSE)
            
            SearchResultsData=SearchResultsData.append(pd.DataFrame(data=[[TrialNumber, str(batch_size_trial)+'-'+str(epochs_trial), RMSE]],
                                                                    columns=['TrialNumber', 'Parameters', 'RMSE'] ))
    return(SearchResultsData)


######################################################
# Calling the function
ResultsData=FunctionFindBestParams(X_train, y_train, X_test, y_test)

1 Parameters: batch_size: 10 - epochs: 5 RMSE: 2.6043818
2 Parameters: batch_size: 10 - epochs: 10 RMSE: 2.6118038
3 Parameters: batch_size: 10 - epochs: 15 RMSE: 2.6043818
4 Parameters: batch_size: 15 - epochs: 5 RMSE: 2.6092937
5 Parameters: batch_size: 15 - epochs: 10 RMSE: 2.6068196
6 Parameters: batch_size: 15 - epochs: 15 RMSE: 2.5996144
7 Parameters: batch_size: 20 - epochs: 5 RMSE: 2.6043818
8 Parameters: batch_size: 20 - epochs: 10 RMSE: 2.5972848
9 Parameters: batch_size: 20 - epochs: 15 RMSE: 2.5996144
10 Parameters: batch_size: 50 - epochs: 5 RMSE: 2.6068196
11 Parameters: batch_size: 50 - epochs: 10 RMSE: 2.6118038
12 Parameters: batch_size: 50 - epochs: 15 RMSE: 2.6143494
13 Parameters: batch_size: 100 - epochs: 5 RMSE: 2.6068196
14 Parameters: batch_size: 100 - epochs: 10 RMSE: 2.6068196
15 Parameters: batch_size: 100 - epochs: 15 RMSE: 2.6143494
16 Parameters: batch_size: 200 - epochs: 5 RMSE: 2.6068196
17 Parameters: batch_size: 200 - epochs: 10 RMSE: 2.6043818
18 Para